# Introduction

This notebook presents an informal introduction to geometric algebra in Dyalog APL. Distinctions are not made between theoretical algebra concepts and its implementation in APL.

#### Disclaimer

Some knowledge of linear algebra and vector spaces could be handy, but all concepts will be introduced. However, bear in mind that this is not a formal text. The definitions are not rigorous and properties are used before being defined. Some links are provided, but the linked content might differ from the content presented here.

Refer to a proper book for a deeper understanding of geometric (or Clifford) algebras. Other introductions and additional resources can also be found online. Some suggestions:

- [David Hestenes web site](https://geocalc.clas.asu.edu/)
- [Alan Macdonald books](http://www.faculty.luther.edu/~macdonal/#geometric-algebra)
- [Rigid geometric algebra wiki](https://rigidgeometricalgebra.org/)
- [biVector.net](https://bivector.net/)

Code fragments in this notebook are written in [Dyalog APL](https://www.dyalog.com/dyalog/index.htm) ([v18.2](https://www.dyalog.com/dyalog/dyalog-versions/182.htm)) and they are an essential part of this document, including the comments. Therefore, being able to read APL is necessary. Nevertheless, all functions and operators are explained with enough detail to (hopefully) be understood with only a basic knowledge of the language.

## Preliminaries

In [1]:
)clear
⎕IO ← 0        ⍝ important
⎕PP ← 4        ⍝ just for display
⎕CT ← 1e¯10    ⍝ avoid rounding issues
'H'⎕CY'dfns'   ⍝ dfns' quaternions for comparison
Assert ← {⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←1}   ⍝ Assert by Roger Hui, but returns 1
Table  ← ≡{1<d←|⍺⍺ ⍵: ((d-1)⍨)∇∇(⍵⍵¨) ⍵ ⋄ ⍵⍵ ⍵}⍕¨(,[¯0.5])   ⍝ display as table

clear ws

# Vector spaces

A [vector space](https://en.wikipedia.org/wiki/Vector_space) is a set of elements which follow certain properties.

## Vector space properties

The `VS` function will tell us if the three values given as right argument, called *[vectors](https://en.wikipedia.org/wiki/Vector_(mathematics_and_physics))*, together with the two values given as left argument, called *[scalars](https://en.wikipedia.org/wiki/Scalar_(mathematics))*, staisfy the properties of vector spaces:

In [2]:
]dinput
VS ← {                                         ⍝ (V)ector (S)pace
    _ ← Assert 0 = ≢∘⍴¨ a b ← ⍺                ⍝ a and b are scalars (no shape)
    _ ← Assert 1 = ⍴∘⍴¨ u v w ← ⍵              ⍝ u, v and w are vectors (rank 1 arrays)
    _ ← Assert 1 = ≢∪≢¨ u v w                  ⍝ u, v and w have the same length
                                               ⍝ properties:
    _ ← Assert v ≡ v + z ← ≠⍨ v                ⍝   identity element for addition (zeros)
    _ ← Assert v ≡ v × 1                       ⍝   identity element for multiplication (1)
    _ ← Assert z ≡ v + -v                      ⍝   inverse element for addition
    _ ← Assert (v + w) ≡ w + v                 ⍝   commutative addition
    _ ← Assert (a × v) ≡ v × a                 ⍝   commutative product with scalars
    _ ← Assert (u + v + w) ≡ (u + v) + w       ⍝   associative addition
    _ ← Assert (v × a × b) ≡ (v × a) × b       ⍝   associative product with scalars
    _ ← Assert (a × v + w) ≡ (a × v) + a × w   ⍝   distributive over addition of vectors
        Assert (v × a + b) ≡ (v × a) + v × b   ⍝   distributive over addition of scalars
}

Let's generate some sample data (two scalars and three 3D vectors) and check that they fulfill these properties:

In [3]:
'ab'  Table a b   ←  5×?  2⍴0   ⍝ 2 random scalars
'uvw' Table u v w ← ↓5×?3 3⍴0   ⍝ 3 random 3D vectors
a b VS u v w                    ⍝ vector space?

┌─────┬────┐
│a │b │
├─────┼────┤
│1.365│4.72│
└─────┴────┘

┌──────────────────┬─────────────────┬─────────────────┐
│u │v │w │
├──────────────────┼─────────────────┼─────────────────┤
│0.04598 1.703 4.05│3.727 1.873 1.489│1.292 3.005 2.559│
└──────────────────┴─────────────────┴─────────────────┘

Vectors and scalars in APL follow the properties of vector spaces. This is a direct consequence of the application of the concepts of [scalar function](https://aplwiki.com/wiki/Scalar_function) and [scalar extension](https://aplwiki.com/wiki/Scalar_extension).

## Nested vectors

APL is an array language, so it can work with several vectors at the same time. There are various ways in which we could choose to represent multiple vectors. We will do it always using rank 1 arrays, but where the elements can be [nested arrays](https://aplwiki.com/wiki/Nested_array). We will call the vectors with non-scalar components *nested vectors*, and each of the vectors that a nested vector contains an *element array*.

For example, we could represent multiple vectors as "vectors of vectors", in which each component (each element array) is a rank 1 array too (we also define an `RD` function to make it easier to generate some random data):

In [4]:
RD ← {5×?⍵⍴0}   ⍝ generate ⍵ values with (R)andom (D)ata
⍝ eg
'ab' Table a b ← ⊂∘RD¨2⍴5   ⍝ 2 random scalars with 5 elements each
'u'  Table ,⊂u ←   RD¨3⍴5   ⍝ random 3D vector with 5 elements by component
'v'  Table ,⊂v ←   RD¨3⍴5   ⍝ random 3D vector with 5 elements by component
'w'  Table ,⊂w ←   RD¨3⍴5   ⍝ random 3D vector with 5 elements by component
a b VS u v w                ⍝ vector space?

┌───────────────────────────────┬───────────────────────────────┐
│a │b │
├───────────────────────────────┼───────────────────────────────┤
│┌─────────────────────────────┐│┌─────────────────────────────┐│
││1.726 3.416 1.155 1.086 3.806│││1.937 3.908 2.255 3.754 4.228││
│└─────────────────────────────┘│└─────────────────────────────┘│
└───────────────────────────────┴───────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│u │
├─────────────────────────────────────────────────────────────────────────────────────────────┤
│┌─────────────────────────────┬──────────────────────────────┬──────────────────────────────┐│
││2.208 2.692 2.736 1.112 1.372│0.7806 1.462 4.561 4.982 1.161│2.406 3.663 0.9834 1.656 2.263││
│└─────────────────────────────┴──────────────────────────────┴──────────────────────────────┘│
└─────────────────────────────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────────────────────────────┐
│v │
├──────────────────────────────────────────────────────────────────────────────────────────┤
│┌────────────────────────────┬─────────────────────────────┬─────────────────────────────┐│
││1.807 3.255 3.955 4.242 4.54│4.972 1.473 3.004 2.051 4.781│4.008 4.167 4.737 3.416 2.277││
│└────────────────────────────┴─────────────────────────────┴─────────────────────────────┘│
└──────────────────────────────────────────────────────────────────────────────────────────┘

┌────────────────────────────────────────────────────────────────────────────────────────────┐
│w │
├────────────────────────────────────────────────────────────────────────────────────────────┤
│┌─────────────────────────────┬───────────────────────────┬────────────────────────────────┐│
││2.515 1.414 3.612 4.967 3.383│2.508 3.509 3.878 3.046 2.4│0.06021 2.226 3.278 0.7429 3.816││
│└─────────────────────────────┴───────────────────────────┴────────────────────────────────┘│
└────────────────────────────────────────────────────────────────────────────────────────────┘

The vector space properties stand for APL nested vectors too. Of course, only as long as their elements [conform](https://aplwiki.com/wiki/Conformability). Vectors and scalars must conform too. Therefore, we need to enclose `a` and `b` to actually be scalars, and the `u`, `v` and `w` vectors must be rank-1 arrays of the same length:

In [5]:
(,'⍴' '⍴¨'∘.,'abuvw') Table (⍴¨,⍴¨¨)a b u v w
⍝ checks in VS:
Assert (⊂⍬) ≡∘⍴¨ a b        ⍝ a and b are scalars
Assert (⊂,1) ≡∘⍴∘⍴¨ u v w   ⍝ u, v and w are vectors (rank 1 arrays)
Assert 2=/≢¨ u v w          ⍝ u, v and w have the same length

┌──┬──┬──┬──┬──┬───┬───┬───────┬───────┬───────┐
│⍴a│⍴b│⍴u│⍴v│⍴w│⍴¨a│⍴¨b│⍴¨u │⍴¨v │⍴¨w │
├──┼──┼──┼──┼──┼───┼───┼───────┼───────┼───────┤
│ │ │3 │3 │3 │┌─┐│┌─┐│┌─┬─┬─┐│┌─┬─┬─┐│┌─┬─┬─┐│
│ │ │ │ │ ││5│││5│││5│5│5│││5│5│5│││5│5│5││
│ │ │ │ │ │└─┘│└─┘│└─┴─┴─┘│└─┴─┴─┘│└─┴─┴─┘│
└──┴──┴──┴──┴──┴───┴───┴───────┴───────┴───────┘

Other representations of multiple vectors are also possible. Just to display them in a convenient way, we will use "vectors of columns" (and define an `RC` function analogous to `RD` to generate random columns):

In [6]:
RC ← (⍪RD)¨⍴   ⍝ generate ⍺ (R)andom (C)olumns of ⍵ rows
⍝ eg
'ab'  Table a b   ← ⊂¨ 2 RC 5   ⍝ 2 random scalar columns of 5 rows
'uvw' Table u v w ← ↓3 3 RC 5   ⍝ 3 random 3D vectors of 5 rows
a b VS u v w                    ⍝ vector space?

┌────────┬────────┐
│a │b │
├────────┼────────┤
│┌──────┐│┌──────┐│
││3.309 │││0.4005││
││4.118 │││1.996 ││
││2.876 │││4.992 ││
││4.699 │││4.221 ││
││0.3687│││4.918 ││
│└──────┘│└──────┘│
└────────┴────────┘

┌───────────────────────┬──────────────────────┬─────────────────────┐
│u │v │w │
├───────────────────────┼──────────────────────┼─────────────────────┤
│┌───────┬─────┬───────┐│┌──────┬──────┬──────┐│┌──────┬─────┬──────┐│
││1.017 │4.525│1.343 │││4.991 │0.4499│0.176 │││1.933 │3.138│2.826 ││
││4.983 │2.363│2.48 │││3.075 │4.298 │1.162 │││3.563 │3.569│2.867 ││
││1.231 │1.515│3.193 │││2.979 │3.19 │0.823 │││4.907 │1.87 │4.515 ││
││2.557 │2.34 │0.08875│││0.4763│0.2869│0.3809│││0.7518│1.249│0.9148││
││0.04726│4.815│4.971 │││0.9872│2.65 │3.207 │││4.902 │1.269│3.883 ││
│└───────┴─────┴───────┘│└──────┴──────┴──────┘│└──────┴─────┴──────┘│
└───────────────────────┴──────────────────────┴─────────────────────┘

Vectors of columns form a vector space too. The shape is the same as before, and the elements conform:

In [7]:
(,'⍴' '⍴¨'∘.,'abuvw') Table (⍴¨,⍴¨¨)a b u v w

┌──┬──┬──┬──┬──┬─────┬─────┬─────────────┬─────────────┬─────────────┐
│⍴a│⍴b│⍴u│⍴v│⍴w│⍴¨a │⍴¨b │⍴¨u │⍴¨v │⍴¨w │
├──┼──┼──┼──┼──┼─────┼─────┼─────────────┼─────────────┼─────────────┤
│ │ │3 │3 │3 │┌───┐│┌───┐│┌───┬───┬───┐│┌───┬───┬───┐│┌───┬───┬───┐│
│ │ │ │ │ ││5 1│││5 1│││5 1│5 1│5 1│││5 1│5 1│5 1│││5 1│5 1│5 1││
│ │ │ │ │ │└───┘│└───┘│└───┴───┴───┘│└───┴───┴───┘│└───┴───┴───┘│
└──┴──┴──┴──┴──┴─────┴─────┴─────────────┴─────────────┴─────────────┘

In some situations, it can be useful to represent multiple vectors not as a nested vector, but as an array of vectors, with the shape of the element arrays. The function `AV` takes a nested vector and returns an array of vectors, while `VA` performs the opposite operation:

In [8]:
AV ← ⊂⍤ 1↑[0]   ⍝ (A)rray of (V)ectors from vector of arrays
VA ← ⊂⍤¯1↑[0]   ⍝ (V)ector of (A)rrays from array of vectors
Assert v ≡ VA AV v
'v' '⍴v' '⍴¨v' 'AV v' '⍴AV v' '⍴¨AV v' Table v (⍴v) (⍴¨v) (AV v) (⍴AV v) (⍴¨AV v)

┌──────────────────────┬──┬─────────────┬──────────────────────┬─────┬──────┐
│v │⍴v│⍴¨v │AV v │⍴AV v│⍴¨AV v│
├──────────────────────┼──┼─────────────┼──────────────────────┼─────┼──────┤
│┌──────┬──────┬──────┐│3 │┌───┬───┬───┐│┌────────────────────┐│5 1 │┌─┐ │
││4.991 │0.4499│0.176 ││ ││5 1│5 1│5 1│││4.991 0.4499 0.176 ││ ││3│ │
││3.075 │4.298 │1.162 ││ │└───┴───┴───┘│├────────────────────┤│ │├─┤ │
││2.979 │3.19 │0.823 ││ │ ││3.075 4.298 1.162 ││ ││3│ │
││0.4763│0.2869│0.3809││ │ │├────────────────────┤│ │├─┤ │
││0.9872│2.65 │3.207 ││ │ ││2.979 3.19 0.823 ││ ││3│ │
│└──────┴──────┴──────┘│ │ │├────────────────────┤│ │├─┤ │
│ │ │ ││0.4763 0.2869 0.3809││ ││3│ │
│ │ │ │├────────────────────┤│ │├─┤ │
│ │ │ ││0.9872 2.65 3.207 ││ ││3│ │
│ │ │ │└────────────────────┘│ │└─┘ │
└──────────────────────┴──┴─────────────┴──────────────────────┴─────┴──────┘

## Scalar extension

If we are working with nested vectors, it could often happen that all the elements of some element array have the same value. For instance, a component could take the value zero if all vectors are contained in a plane. In these cases, we do not need to store multiple copies of the same value. After all, APL will use scalar extension if we have some scalar component:

In [9]:
u[1]   ← 0            ⍝ vectors parallel to plane xz
w[0 2] ← 0 1          ⍝ vectors with same x and z components
'uvw' Table u v w     ⍝ vectors with scalar components
a b VS u v w          ⍝ vector space?

┌───────────────────┬──────────────────────┬───────────┐
│u │v │w │
├───────────────────┼──────────────────────┼───────────┤
│┌───────┬─┬───────┐│┌──────┬──────┬──────┐│┌─┬─────┬─┐│
││1.017 │0│1.343 │││4.991 │0.4499│0.176 │││0│3.138│1││
││4.983 │ │2.48 │││3.075 │4.298 │1.162 │││ │3.569│ ││
││1.231 │ │3.193 │││2.979 │3.19 │0.823 │││ │1.87 │ ││
││2.557 │ │0.08875│││0.4763│0.2869│0.3809│││ │1.249│ ││
││0.04726│ │4.971 │││0.9872│2.65 │3.207 │││ │1.269│ ││
│└───────┴─┴───────┘│└──────┴──────┴──────┘│└─┴─────┴─┘│
└───────────────────┴──────────────────────┴───────────┘

Vectors with mixed array and scalar components also follow the properties of vector spaces. Some components have different shapes, but all of them conform:

In [10]:
(,'⍴' '⍴¨'∘.,'uvw') Table (⍴¨,⍴¨¨)u v w

┌──┬──┬──┬──────────┬─────────────┬───────┐
│⍴u│⍴v│⍴w│⍴¨u │⍴¨v │⍴¨w │
├──┼──┼──┼──────────┼─────────────┼───────┤
│3 │3 │3 │┌───┬┬───┐│┌───┬───┬───┐│┌┬───┬┐│
│ │ │ ││5 1││5 1│││5 1│5 1│5 1││││5 1│││
│ │ │ │└───┴┴───┘│└───┴───┴───┘│└┴───┴┘│
└──┴──┴──┴──────────┴─────────────┴───────┘

Vectors with scalar components of course are smaller:

In [11]:
]SizeOf u v w

v 296 u 248 w 200

In some cases, this space savings could be significant. However, either keeping the size low when working with the vector or simplifying it later will have a performance cost. In following sections, we will study the performance and memory usage implications of vectors with scalar components with more detail.

To make it easier working with vectors with scalar components, we define the collapse function, `C`, and the zero operator, `_Z`. The `C` function will colapse components of nested vectors into scalars if all elements are equal. The `⎕CT` value used in the comparison can be passed as left argument. Moreover, the function makes sure that all values for which `⎕CT>|` is true are treated as zero (in general, APL will never consider identical two values when one of them is `0`, independently of `⎕CT`). The `_Z` operator is intended to be used with dyadic functions for which a zero argument implies a result of zero, such as multiplication. It checks if any of the arguments is a scalar `0` and, in that case, it returns a scalar `0`. This way we avoid having to run `C` later, which is more expensive. Consequently, the performance is improved when using vectors with scalar zeros. It is still lower than when not using `C` or `_Z` at all, but the space requirements will be smaller:

In [12]:
 C  ← {⍺←⎕CT ⋄ ⎕CT←⍺ ⋄ 1=≢∪⍵:⊃⍵ ⋄ ⍺∧.>|∊⍵:0 ⋄ ⍵}¨   ⍝ (C)ollapse identical values into scalar
_Z  ← {0≡⍺:0 ⋄ 0≡⍵:0 ⋄ ⍺(⍺⍺)⍵}                      ⍝ result (Z)ero for zero argument
⍝ eg
v1 v2 ← ((1 2) 0 (3 4) (5 6)) ((7 8) (9 10) (11 12) 0)
'v1' 'v2' 'v1 × v2' 'v1 ×_Z¨ v2' Table v1 v2 (v1 × v2) (v1 ×_Z¨ v2)
Assert 0 1 2 3   ≡ C 0 1 (2 2 2) 3
Assert (0 1) 0 ≡ (0 1) (2 3) ×_Z¨ 1 0
]Runtime '  u ×    w' '  u ×_Z¨ w' 'C u ×    w' -compare
]Runtime 'r1←  u ∘.×¨   w' 'r2←  u ∘.×_Z¨ w' 'r3←C u ∘.×¨   w' -compare
'r1' 'r2' 'r3' Table r1 r2 r3
]SizeOf r1 r2 r3

┌───────────────┬──────────────────┬────────────────────┬────────────────┐
│v1 │v2 │v1 × v2 │v1 ×_Z¨ v2 │
├───────────────┼──────────────────┼────────────────────┼────────────────┤
│┌───┬─┬───┬───┐│┌───┬────┬─────┬─┐│┌────┬───┬─────┬───┐│┌────┬─┬─────┬─┐│
││1 2│0│3 4│5 6│││7 8│9 10│11 12│0│││7 16│0 0│33 48│0 0│││7 16│0│33 48│0││
│└───┴─┴───┴───┘│└───┴────┴─────┴─┘│└────┴───┴─────┴───┘│└────┴─┴─────┴─┘│
└───────────────┴──────────────────┴────────────────────┴────────────────┘

u × w → 3.9E¯7 | 0% ⎕⎕ 
* u ×_Z¨ w → 2.1E¯6 | +438% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* C u × w → 8.5E¯6 | +2117% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

r1← u ∘.×¨ w → 1.2E¯6 | 0% ⎕⎕⎕⎕⎕ 
* r2← u ∘.×_Z¨ w → 2.5E¯6 | +103% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* r3←C u ∘.×¨ w → 9.1E¯6 | +630% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌─────────────┬─────────────┬─────────────┐
│r1 │r2 │r3 │
├─────────────┼─────────────┼─────────────┤
│┌─┬─┬───────┐│┌─┬─┬───────┐│┌─┬─┬───────┐│
││0│0│1.343 │││0│0│1.343 │││0│0│1.343 ││
││0│0│2.48 │││ │ │2.48 │││ │ │2.48 ││
││0│0│3.193 │││ │ │3.193 │││ │ │3.193 ││
││0│0│0.08875│││ │ │0.08875│││ │ │0.08875││
││0│0│4.971 │││ │ │4.971 │││ │ │4.971 ││
│└─┴─┴───────┘│└─┴─┴───────┘│└─┴─┴───────┘│
└─────────────┴─────────────┴─────────────┘

r1 232 r2 200 r3 200

## Vector extension

Actually, there is no reason to require vectors to be of the same length to do operations with them. A vector of `n` dimensions can always be interpreted as a vector of `n+1` dimensions in which the additional component is `0`. Therefore, it can also be interpreted as a vector of `n+2` components, of `n+3` components, and so on. However, if we try to operate with vectors of different lengths in APL, we will get an error:

In [13]:
1 2 3 + 4 5   ⍝ ERROR: length

LENGTH ERROR: Mismatched left and right argument shapes
      1 2 3+4 5   ⍝ ERROR: length
           ∧


We are going define some functions and operators to make it easier to work with vectors of different lengths. We want to be able to extend the arguments of functions to be of the required length, appending as many (scalar) zeros as needed, and also to remove trailing zeros when they are not necessary.

In [14]:
 X  ← {⍺↑⍵,0⍴⍨0⌈⍺-≢⍵}                        ⍝ e(X)tend vector ⍵ to length ⍺ appending scalar zeros
_X_ ← {⍺←⊢ ⋄ ⍺ ⍺⍺⍥((⍺ ⍵⍵⍨⍨ ⍵)∘X)⍵}           ⍝ e(X)tend ⍺⍺ arguments to have length given by ⍵⍵
_X  ← _X_(⌈⍥≢)                               ⍝ e(X)tend ⍺⍺ arguments to have same (maximum) length
_ZX ← {⍺ ⍺⍺_Z¨_X_(⌊⍥≢)⍵}                     ⍝ e(X)tend ⍺⍺ arguments to have same (minimum) length and _Z¨
 S  ← {⍬≡⍵:0 ⋄ (1<≢⍵)∨(0<≡⍵)∧1<≢⊃⍵:⍵ ⋄ ⊃⍵}   ⍝ return (S)calar as scalar
 Z  ← {⍺←⎕CT ⋄ ⎕CT←⍺ ⋄ S⍵↓⍨-⊥⍨(1∧.=1+∊)¨⍵}   ⍝ remove trailing (Z)eros
⍝ eg
Assert 0 1 2 0   ≡ 4 X 0 1 2
Assert 0 1       ≡ 2 X 0 1 2
Assert 0 1 2     ≡ Z 0 1 2 (0 0 0) 0
Assert 0         ≡ Z 0 0 0
Assert 1         ≡ Z 1 0
Assert 0 1 1     ≡ 0 0 1 + _X 0 1
Assert 1 0 2     ≡ Z 1 0 2 0 0
Assert 1 0 2     ≡ 1 0 2 3 ×_ZX 1 1 1

The function `X` extends a vector with scalar zeros up to the given length. The `_X_` operator applies the given function after extending the arguments to the length specified, either explicitly or as the result of a function. For instance, the monadic operator `_X` extend the arguments up to the maximum length of both of them (`⌈⍥≢`), while `_ZX` will "extend" them to the minimum length and apply the given function using `_Z¨`. The functions `S` and `Z` are used to simplify vectors: `S` will return 1-element vectors as scalars and 0-elements vectors as `0`, while `Z` will remove trailing zeros (like `C`, using `⎕CT` or a left argument), including non-scalar components in which all elements are zero.

When the arguments of a function are first extended to the required length, and later the result is simplified collapsing into scalars and removing trailing zeros, we will say that the function is applied using *vector extension*. The operator `_V` applies a function using vector extension.

In [15]:
_V ← {⍺←⊢ ⋄ Z∘C ⍺(⍺⍺ _X)⍵}   ⍝ (V)ector extension
⍝ eg
v1 v2 ← (1 2 3) (4 5)
'v1' 'v2' 'v1 +_V v2' Table v1 v2 (v1 +_V v2)           ⍝ no error now
v1 v2 ← ((⍪v1) 0 0 (⍪v1) 0) ((⍪v2,6) (⍪v2,6) 0 (⍪v1))
'v1' 'v2' 'v1 -_V v2' Table v1 v2 (v1 -_V v2)           ⍝ subtraction under vector extension
Assert 0 (¯3 ¯4) ≡ 0 0 0 (1 2) -_V 0 (3 4) 0 (1 2)

┌─────┬───┬─────────┐
│v1 │v2 │v1 +_V v2│
├─────┼───┼─────────┤
│1 2 3│4 5│5 7 3 │
└─────┴───┴─────────┘

┌───────────┬─────────┬─────────┐
│v1 │v2 │v1 -_V v2│
├───────────┼─────────┼─────────┤
│┌─┬─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌──┬──┐ │
││1│0│0│1│0│││4│4│0│1│││¯3│¯4│ │
││2│ │ │2│ │││5│5│ │2│││ │¯5│ │
││3│ │ │3│ │││6│6│ │3│││ │¯6│ │
│└─┴─┴─┴─┴─┘│└─┴─┴─┴─┘│└──┴──┘ │
└───────────┴─────────┴─────────┘

## Vector spaces under vector extension

Now that we can operate with vectors of arbitrary lengths, we can redefine the `VS` function, removing the requirement for vectors to have the same length. We also define an `E` function to check that two vectors are *equivalent* or, in other words, that they match after being simplified by `Z∘C`.

In [16]:
E ← ≡⍥(Z∘C)   ⍝ (E)quivalent vectors

In [17]:
]dinput
VS ← {                                             ⍝ (V)ector (S)pace
    _ ← Assert (⊂⍬) ∧.≡ ⍴¨ a b ← ⍺                 ⍝ a and b are scalars
    _ ← Assert (⊂,1) ∧.≥ ⍴∘⍴¨ u v w ← ⍵            ⍝ u, v and w are vectors (rank 1)
    _ ← Assert 0 = ≢∘⍴¨ a b ← ⍺                    ⍝ a and b are scalars (no shape)
    _ ← Assert 1 = ⍴∘⍴¨ u v w ← ⍵                  ⍝ u, v and w are vectors (rank 1 arrays)
                                                   ⍝ properties:
    _ ← Assert v                 E v + 0           ⍝   identity element for addition
    _ ← Assert v                 E 1 × v           ⍝   identity element for multiplication
    _ ← Assert 0                 E v +_V -v        ⍝   inverse element for addition
    _ ← Assert (v +_V w)         E w +_V v         ⍝   commutative addition
    _ ← Assert (a × v)           E v × a           ⍝   commutative product with scalars
    _ ← Assert (u +_V v +_V w)   E w +_V u +_V v   ⍝   associative addition
    _ ← Assert (v × a × b)       E (v × a) × b     ⍝   associative product with scalars
    _ ← Assert (a × v +_V w)     E v +_V⍥(a∘×) w   ⍝   distributive over addition of vectors
        Assert (v × a + b)       E a +_V⍥(v∘×) b   ⍝   distributive over addition of scalars
}

In [18]:
a b VS 1 (0 1) (0 0 1)

## Magnitude and unitary vector

Let's define some more functions to work with vectors. The function `M` calculates the magnitude of a vector using the dot product, while `U` returns a unitary vector parallel to the given one multiplying it by the inverse of `M`.

In [19]:
M ← 0.5*⍨+.×⍨   ⍝ (M)agnitude of vector
U ← ×∘(÷M)⍨     ⍝ (U)nitary vector
⍝ eg
'M v' 'U v' 'M w' 'U w' Table (M v) (U _V v) (M w) (U _V w)
Assert E∘(M×U)⍨¨ u v w   ⍝ a vector is the product of its magnitude by its unit vector

┌───────┬───────────────────────┬───────┬─────────────────┐
│M v │U v │M w │U w │
├───────┼───────────────────────┼───────┼─────────────────┤
│┌─────┐│┌──────┬───────┬──────┐│┌─────┐│┌─┬──────┬──────┐│
││5.014│││0.9953│0.08973│0.0351│││3.293│││0│0.9528│0.3036││
││5.412│││0.5683│0.7943 │0.2147│││3.707│││ │0.9629│0.2698││
││4.442│││0.6708│0.7182 │0.1853│││2.121│││ │0.8819│0.4715││
││0.674│││0.7067│0.4257 │0.5651│││1.6 │││ │0.7806│0.625 ││
││4.275│││0.2309│0.6199 │0.75 │││1.616│││ │0.7855│0.6188││
│└─────┘│└──────┴───────┴──────┘│└─────┘│└─┴──────┴──────┘│
└───────┴───────────────────────┴───────┴─────────────────┘

# Geometric product

## Inverse of a vector

In addition to the properties of vector spaces, in a geometric algebra, there is an inverse element for multiplication using the geometric product (which has not been defined yet) such that, for every vector `v` not equivalent to zero, it exists an inverse that multiplied by the original vector results in `1`.

It can help at this point to think of a concrete example to get an intuitive idea of what this inverse should be. Let's consider, for instance, a velocity, a scalar velocity. Let’s say this velocity is $5\,m/s$. There is no discussion in that the inverse of this velocity is $0.2\,s/m$. Similarly, a velocity of $-5\,m/s$ will have an inverse of $-0.2\,s/m$. The direction of the inverse is the same as the original, but its magnitude is the inverse of the original magnitude. Another way to express this is that the inverse is the original value divided by the square of its magnitude (which, for scalars, is just its self product, `×⍨`):

In [20]:
Assert ((|∘÷ ≡ ÷∘|),(×∘÷ ≡ ×),(÷ ≡ ÷∘(×⍨)⍨))5 ¯5
' ' '÷' '÷∘(×⍨)⍨' '×∘÷⍨' Table (5 ¯5) (÷5 ¯5) (÷∘(×⍨)⍨5 ¯5) (×∘÷⍨5 ¯5)

┌────┬────────┬────────┬────┐
│ │÷ │÷∘(×⍨)⍨ │×∘÷⍨│
├────┼────────┼────────┼────┤
│5 ¯5│0.2 ¯0.2│0.2 ¯0.2│1 1 │
└────┴────────┴────────┴────┘

Our intention is to extend this concept to vectors. Therefore, we define the inverse of any vector `v` not equivalent to zero (`0(~E)v`) as another vector with the same direction of `v`, but with a magnitude which is the inverse of the magnitude of `v`. As with scalars, we can achieve this result dividing the original vector by the square of its magnitude (which, for vectors, is just its self dot product, `+.×⍨`):

In [21]:
I ← ×∘(÷+.×⍨)⍨   ⍝ (I)nverse vector
⍝ eg
'I v' '+.×∘I⍨ v' 'I w' '+.×∘I⍨ w' Table (I _V v) (+.×∘I⍨ v) (I _V w) (+.×∘I⍨ w) 
Assert (M I v) ≡ ÷ M v        ⍝ magnitude of inverse is inverse of magnitude
Assert (U I v) ≡ U v          ⍝ same direction
Assert   (I v) ≡ ((÷M)×U) v   ⍝ inverse vector
⍝ the inverse of v is v divided by the square of its magnitude:
Assert   (I v) ≡ ÷∘(×⍨M)⍨ v   ⍝ analogous to (÷⍵) ≡ ÷∘(×⍨|)⍨⍵

┌─────────────────────────┬────────┬──────────────────┬────────┐
│I v │+.×∘I⍨ v│I w │+.×∘I⍨ w│
├─────────────────────────┼────────┼──────────────────┼────────┤
│┌──────┬───────┬────────┐│┌─┐ │┌─┬──────┬───────┐│┌─┐ │
││0.1985│0.01789│0.006999│││1│ ││0│0.2893│0.0922 │││1│ │
││0.105 │0.1468 │0.03968 │││1│ ││ │0.2598│0.07278│││1│ │
││0.151 │0.1617 │0.04171 │││1│ ││ │0.4158│0.2223 │││1│ │
││1.048 │0.6315 │0.8384 │││1│ ││ │0.4879│0.3906 │││1│ │
││0.054 │0.145 │0.1754 │││1│ ││ │0.4861│0.383 │││1│ │
│└──────┴───────┴────────┘│└─┘ │└─┴──────┴───────┘│└─┘ │
└─────────────────────────┴────────┴──────────────────┴────────┘

This definition might look a bit arbitrary, and indeed it is up to some point. We will come back to it later.

Notice that the `I` function does not directly uses the dyadic divide function because, if we defined `I` as `÷∘(+.×⍨)⍨`, then when we had a vector equivalent to zero, we would be dividing `0` by `0` and, in APL, `1 ≡ 0÷0`. However `÷0` results in a domain error, which looks like a more reasonable response. It is also important to be careful with nested vectors, since the function will fail with a single vector equivalent to `0` (when `0(~E)¨AV`):

In [22]:
Assert 0(~E)¨AV v    ⍝ v has an inverse
vz←v ⋄ (3⌷¨vz) ← 0   ⍝ vz is z but with a vector equivalent to zero
(⊂'vz') Table ,⊂vz
Assert 0(~E)¨AV vz   ⍝ FAIL
I vz                 ⍝ ERROR: domain

┌─────────────────────┐
│vz │
├─────────────────────┤
│┌──────┬──────┬─────┐│
││4.991 │0.4499│0.176││
││3.075 │4.298 │1.162││
││2.979 │3.19 │0.823││
││0 │0 │0 ││
││0.9872│2.65 │3.207││
│└──────┴──────┴─────┘│
└─────────────────────┘

assertion failure
      Assert 0(~E)¨AV vz   ⍝ FAIL
      ∧
DOMAIN ERROR: Divide by zero
      I vz                 ⍝ ERROR: domain
      ∧


## Geometric product of vectors

Once vector inverses are defined, the geometric product of vectors (represented here by `∆V`) is the associative and distributive product for which a vector multiplied by its inverse is equal to one:

    1 ≡ v ∆V I v  ⍝ ∆V is the geometric product of vectors
    
From this definition, several properties can be derived. Before properly defining the `∆V` function, let's first see what is the result of multiplying parallel and perpendicular vectors. We will later use these results to define arbitrary geometric products.

### Geometric product of parallel vectors

When we calculate the inverse of a unitary vector using our definition, we will find that its inverse is the same vector. We divide by 1 when dividing by its magnitude (or its squared magnitude), leaving the original vector unchanged:

In [23]:
Assert ≡∘I⍨ U v
'U v' 'I U v' Table (U v) (I U v)

┌───────────────────────┬───────────────────────┐
│U v │I U v │
├───────────────────────┼───────────────────────┤
│┌──────┬───────┬──────┐│┌──────┬───────┬──────┐│
││0.9953│0.08973│0.0351│││0.9953│0.08973│0.0351││
││0.5683│0.7943 │0.2147│││0.5683│0.7943 │0.2147││
││0.6708│0.7182 │0.1853│││0.6708│0.7182 │0.1853││
││0.7067│0.4257 │0.5651│││0.7067│0.4257 │0.5651││
││0.2309│0.6199 │0.75 │││0.2309│0.6199 │0.75 ││
│└──────┴───────┴──────┘│└──────┴───────┴──────┘│
└───────────────────────┴───────────────────────┘

Also by definition, the geometric product of a vector by its inverse must be 1. This implies that multiplying a unitary vector by itself will also be 1:

    1 ≡ ∆V⍨ U v
    
More generally, we can calculate what is the result of multiplying any vector by itself. If a vector `v` is defined as `m×u`, where `m` is a scalar and `u` is a unitary vector, we can define its square as:

    (∆V⍨ v) ≡ (m × u) ∆V m × u   ⍝ v ≡ m × u
    (∆V⍨ v) ≡ m × m × u ∆V u     ⍝ (a × v) ≡ v × a
    (∆V⍨ v) ≡ m × m              ⍝ 1 ≡ ∆V⍨ u

The result of squaring a vector is the square of its magnitude.

To multiply two parallel vectors `v` and `w` defined, respectively, as `a×u` and `b×u`:

    (v ∆V w) ≡ (a × u) ∆V b × u
    (v ∆V w) ≡ a × b × u ∆V u
    (v ∆V w) ≡ a × b
    
So, the product of two parallel vectors is the scalar that results from multiplying their magnitudes.

### Geometric product of perpendicular vectors

Every vector can be descomposed in two perpendicular components in some base. For example:

In [24]:
v1 v2 ← 3 2   ⍝ eg
Assert (⊃ v1 v2 +.× (1 0) (0 1)) ≡ (M×U) v1 v2   ⍝ ortoghonal components or magnitude and unit vector
Assert 0 ≡ v1 0 +.× 0 v2                         ⍝ (v1 0) and (0 v2) are perpendicular (dot product is zero)
'v' 'v1' 'u1' 'v2' 'u2' Table (3 2) 3 (1 0) 2 (0 1)
'v1 0 +.× 0 v2' 'm ← M v' 'u ← U v' 'm × u' Table (3 0+.×0 2) (M v1 v2) (U v1 v2) ((M×U) v1 v2)

┌───┬──┬───┬──┬───┐
│v │v1│u1 │v2│u2 │
├───┼──┼───┼──┼───┤
│3 2│3 │1 0│2 │0 1│
└───┴──┴───┴──┴───┘

┌─────────────┬───────┬─────────────┬─────┐
│v1 0 +.× 0 v2│m ← M v│u ← U v │m × u│
├─────────────┼───────┼─────────────┼─────┤
│0 │3.606 │0.8321 0.5547│3 2 │
└─────────────┴───────┴─────────────┴─────┘

We saw before that the result of squaring the vector `v` (with magnitude `m`) will be its magnitude squared (`×⍨m`). We must get the same result when using as starting point the decomposition of `v` in perpendicular vectors:

    (∆V⍨ v) ≡ ×⍨m
    (∆V⍨ v) ≡ ((v1×u1) + v2×u2) ∆V (v1×u1) + v2×u2

Developing further this expression, applying the distributive property:

    (∆V⍨ v) ≡ (∆V⍨ v1×u1) + ((v1×u1) ∆V v2×u2) + ((v2×u2) ∆V v1×u1) + ∆V⍨ v2×u2

And, applying the already known square formula and the commutative property for multiplication by scalars:

    (∆V⍨ v) ≡ (×⍨v1) + (v1×v2 × u1 ∆V u2) + (v1×v2 × u2 ∆V u1) + ×⍨v2

The value `∆V⍨ v` is also `×⍨m` which, according to [Pythagoras](https://en.wikipedia.org/wiki/Pythagorean_theorem), must be equal to `(×⍨v1) + ×⍨v2`. Therefore:

    0 ≡ v1×v2 × (u1 ∆V u2) + u2 ∆V u1
    0 ≡ (u1 ∆V u2) + u2 ∆V u1

So:

    (u1 ∆V u2) ≡ -u2 ∆V u1

The product of two perpendicular vectors is anti-commutative. As a particular solution, both `u1 ∆V u2` and `u2 ∆V u1` might be equal to zero. Let's assume they do and see what happens:

    (V u) ≡ (V u) ∆ (V v) ∆ (V v)
    (V u) ≡((V u) ∆ (V v))∆ (V v)
    (V u) ≡             0 ∆ (V v)
    (V u) ≡ 0 ⍝ ???

If we defined the product of orthogonal vectors as zero, we would find that the product we are searching for cannot be associative, as we established in our definition. In fact, this product would be the familiar dot product, `∆V ← +.×`. Instead, in geometric algebras, the associativity of the geometric product will imply that the geometric product of two perpendiculars vectors must be different from zero:

    (0 ≢ u +.× v) ∨ 0 ≢ u ∆V v   ⍝ in other words: u (+.×∨⍥(0∘≢)∆V) v
    
We will call the product of two orthogonal vectors a bivector.

### Bivectors

The value of the product `y ∆V x` (where `y` and `x` are perpendicular vectors) is not a vector, neither a scalar, it is another entity called *[bivector](https://en.wikipedia.org/wiki/Bivector)* (eg `yx ← y ∆V x`). Bivectors can be interpreted as oriented planes, just as vectors can be interpreted as oriented segments.

It can be convenient to consider a more tangible example in order to get an intuitive idea of the geometric properties of bivectors.

We may have, for instance, two perpendicular vectors, each of them representing the width and height of a sheet of paper. To calculate the area of the sheet, we calculate the geometric product `h ∆V w`. Since `h` and `w` are perpendicular, the result will be a bivector (the bivector `hw` in our example). It is easy to prove that we will obtain the same result independently of any rotation that we apply to the sheet of paper.

In [25]:
ps ls←(5 6⍴'h',¯29↑'0    w')(3 9⍴'0',(¯8↑'h'),¯18↑9↑'w')                   ⍝ portrait and landscape sheets
r←' ' '○0.5' '○1' '○1.5' Table ⊂¨ps ls (⌽⊖ps) (⌽⊖ls)
r⊣r⍪←'0 h ∆V w 0' 'h 0 ∆V 0 ¯w' '0 ¯h ∆V ¯w 0' '¯h 0 ∆V 0 w' Table ⊂'hw'   ⍝ rotated

┌──────────┬───────────┬────────────┬───────────┐
│ │○0.5 │○1 │○1.5 │
├──────────┼───────────┼────────────┼───────────┤
│┌──────┐ │┌─────────┐│┌──────┐ │┌─────────┐│
││h │ ││0 h│││w 0│ ││ w││
││ │ ││ │││ │ ││ ││
││ │ ││w │││ │ ││h 0││
││ │ │└─────────┘││ │ │└─────────┘│
││0 w│ │ ││ h│ │ │
│└──────┘ │ │└──────┘ │ │
├──────────┼───────────┼────────────┼───────────┤
│0 h ∆V w 0│h 0 ∆V 0 ¯w│0 ¯h ∆V ¯w 0│¯h 0 ∆V 0 w│
├──────────┼───────────┼────────────┼───────────┤
│hw │hw │hw │hw │
└──────────┴───────────┴────────────┴───────────┘

However, if we flip the sheet, the geometric product will be negative!

In [26]:
f←('⊖',¨' ' '○0.5' '○1' '○1.5') Table ⊂∘⊖¨ps ls (⌽⊖ps) (⌽⊖ls)
f⊣f⍪←'0 ¯h ∆V w 0' 'h 0 ∆V 0 w' '0 h ∆V ¯w 0' '¯h 0 ∆V 0 ¯w' Table ⊂'-hw'  ⍝ flipped vertically

┌───────────┬───────────┬───────────┬────────────┐
│⊖ │⊖○0.5 │⊖○1 │⊖○1.5 │
├───────────┼───────────┼───────────┼────────────┤
│┌──────┐ │┌─────────┐│┌──────┐ │┌─────────┐ │
││0 w│ ││w │││ h│ ││h 0│ │
││ │ ││ │││ │ ││ │ │
││ │ ││0 h│││ │ ││ w│ │
││ │ │└─────────┘││ │ │└─────────┘ │
││h │ │ ││w 0│ │ │
│└──────┘ │ │└──────┘ │ │
├───────────┼───────────┼───────────┼────────────┤
│0 ¯h ∆V w 0│h 0 ∆V 0 w │0 h ∆V ¯w 0│¯h 0 ∆V 0 ¯w│
├───────────┼───────────┼───────────┼────────────┤
│-hw │-hw │-hw │-hw │
└───────────┴───────────┴───────────┴────────────┘

The result is the same if we flip the sheet horizontally instead of vertically:

In [27]:
⍝ figure
r,h←('⌽',¨'○1.5' '○1' '○0.5' ' ')⍪(⊂1+⍳3)⌷2⌽⌽f  ⍝ flipped horizontally

┌──────────┬───────────┬────────────┬───────────┬───────────┬───────────┬────────────┬───────────┐
│ │○0.5 │○1 │○1.5 │⌽○1.5 │⌽○1 │⌽○0.5 │⌽ │
├──────────┼───────────┼────────────┼───────────┼───────────┼───────────┼────────────┼───────────┤
│┌──────┐ │┌─────────┐│┌──────┐ │┌─────────┐│┌─────────┐│┌──────┐ │┌─────────┐ │┌──────┐ │
││h │ ││0 h│││w 0│ ││ w│││w │││0 w│ ││h 0│ ││ h│ │
││ │ ││ │││ │ ││ │││ │││ │ ││ │ ││ │ │
││ │ ││w │││ │ ││h 0│││0 h│││ │ ││ w│ ││ │ │
││ │ │└─────────┘││ │ │└─────────┘│└─────────┘││ │ │└─────────┘ ││ │ │
││0 w│ │ ││ h│ │ │ ││h │ │ ││w 0│ │
│└──────┘ │ │└──────┘ │ │ │└──────┘ │ │└──────┘ │
├──────────┼───────────┼────────────┼───────────┼───────────┼───────────┼────────────┼───────────┤
│0 h ∆V w 0│h 0 ∆V 0 ¯w│0 ¯h ∆V ¯w 0│¯h 0 ∆V 0 w│h 0 ∆V 0 w │0 ¯h ∆V w 0│¯h 0 ∆V 0 ¯w│0 h ∆V ¯w 0│
├──────────┼───────────┼────────────┼───────────┼───────────┼───────────┼────────────┼───────────┤
│hw │hw │hw │hw │-hw │-hw │-hw │-hw │
└──────────┴───────────┴────────────┴───────────┴───────────┴───────────┴────────────┴───────────┘

In this example, a positive area is the one obtained when multiplying height by width. This is an arbitrary choice. More generally, we will impose that the product of a vector of higher dimensions multiplying by the left a vector of lower dimensions is positive. We may as well take the opposite convention and define positive areas as width by height. This would also work but, as it will be clear later, the chosen convention works better with the usual choice of an horizontal axis for only one dimension and APL's right-to-left evaluation rule.

Recall that the product of orthonormal vectors must be anticommutative. Indeed, in our example, flipping the sheet is equivalent to swapping the factors of the product:

In [28]:
(r,h)[;0 4]

┌──────────┬───────────┐
│ │⌽○1.5 │
├──────────┼───────────┤
│┌──────┐ │┌─────────┐│
││h │ ││w ││
││ │ ││ ││
││ │ ││0 h││
││ │ │└─────────┘│
││0 w│ │ │
│└──────┘ │ │
├──────────┼───────────┤
│0 h ∆V w 0│h 0 ∆V 0 w │
├──────────┼───────────┤
│hw │-hw │
└──────────┴───────────┘

### Trivectors

This idea can be further extended to higher dimensions. The product of a bivector `yx` and a perpendicular vector `z` is called a *[trivector](https://en.wikipedia.org/w/index.php?title=Trivector&redirect=no)* (`zyx`). Trivectors can be interpreted as oriented volumes. We will get positive volumes when a positive thickness multiplies a positive area, and negative volumes if one of them is negative. Consequently, a volume is negated when it is turned inside out.

For higher dimensions, the same concepts apply, although it is more difficult to think of intuitive examples.

## n-vectors and multivectors

In general, an *[n-vector](https://en.wikipedia.org/wiki/Multivector)* is defined as the product of a number `n` of linearly independent vectors, and can be interpreted as an n-dimensional entity. So, vectors are *1-vectors*, bivectors are *2-vectors*, trivectors are *3-vectors*, and so on. By extension of this concept, scalars can be considered *0-vectors*.

In general, we will call every element of the algebra, obtained as the product or addition of other elements, from scalars up to n-vectors, a *multivector* of grade `n`, or n-multivector. Scalars will be multivectors of grade 0, and the addition of an n-vector and an m-multivector, with `m≤n`, will be a multivector of grade `n`. Notice that every n-vector is also an n-multivector, but the opposite is not necessarily true. For example, the addition of an scalar and a bivector will be a 2-multivector, but it is not a 2-vector. In summary:

- 0-vector: scalar (also: 0-multivector)
- 1-vector: vector (of any number of dimensions)
- 2-vector: product of 2 orthogonal 1-vectors (bivector)
- 3-vector: product of 3 orthogonal 1-vectors (trivector)
- n-vector of grade `n`: product of `n` orthogonal 1-vectors
- n-multivector of grade `n`: addition of n-vector and m-multivector with `m≤n`

It can be proved that multivectors satisfy the properties of vector spaces. Therefore, we are going to represent multivectors in APL using rank-1 arrays, just as we have been doing with vectors so far.

It will be assumed that the component `n` of an array represents the unit multivector obtained multiplying, from left to right, the set of perpendicular unitary vectors corresponding to the ones in the binary representation of `n` (assuming `⎕IO←0`). Therefore, if the unitary vectors have `n` dimensions, the corresponding multivectors will have `2*n` components, corresponding to all the possible products between the `n` 1-vectors. The unitary multivectors in binary form will be called the *binary unitary base*, or bub.

The `BV` function returns the base multivectors, when given the number of components, and `BB` returns the bub. `G` returns the grade of a multivector. The grade of a multivector is the maximum grade of its components different from zero, and it is found adding the corresponding ones in the bub. Notice that the grade is not the same as the dimensions, given by `D`. While the grade indicates how many 1-vectors are multiplied, the dimensions correspond to the maximum number of dimensions in those 1-vectors. For instance, the vector `z` is a 1-vector of 3 dimensions, and the bivector `yx` is a 2-vector of 2 dimensions. Therefore, for every multivector, `G≤D`.

In [29]:
BV ← 1↑⍨¨∘-1+⍳              ⍝ (B)ase unitary multi(V)ectors for ⍵ components
BB ← ↓∘⍉2⊥⍣¯1⍳              ⍝ (B)inary unitary (B)ase (bub) for ⍵ components
G  ← {⌈/+/¨(0≢¨C ⍵)/BB≢⍵}   ⍝ (G)rade
D  ← ⌈2⍟⌈⍥≢                 ⍝ (D)imensions
⍝ eg: 3D base vectors
d   ← 3                   ⍝ dimensions (3D)
bu  ← BV 2*d              ⍝ base unitary multivectors
bub ← BB 2*d              ⍝ binary unitary base
id  ← '1',1↓bub/¨⊂'zyx'   ⍝ names
'base multivectors' '≢' 'bub' 'grade' 'dim' 'name' Table∘(⍪¨) bu (,∘≢¨bu) bub (,∘G¨bu) (,∘D¨bu) id
Assert (G≤D)¨bu

┌─────────────────┬───┬───────┬─────┬───┬─────┐
│base multivectors│≢ │bub │grade│dim│name │
├─────────────────┼───┼───────┼─────┼───┼─────┤
│┌───────────────┐│┌─┐│┌─────┐│┌─┐ │┌─┐│┌───┐│
││1 │││1│││0 0 0│││0│ ││0│││1 ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 1 │││2│││0 0 1│││1│ ││1│││x ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 1 │││3│││0 1 0│││1│ ││2│││y ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 0 1 │││4│││0 1 1│││2│ ││2│││yx ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 0 0 1 │││5│││1 0 0│││1│ ││3│││z ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 0 0 0 1 │││6│││1 0 1│││2│ ││3│││zx ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 0 0 0 0 1 │││7│││1 1 0│││2│ ││3│││zy ││
│├───────────────┤│├─┤│├─────┤│├─┤ │├─┤│├───┤│
││0 0 0 0 0 0 0 1│││8│││1 1 1│││3│ ││3│││zyx││
│└───────────────┘│└─┘│└─────┘│└─┘ │└─┘│└───┘│
└─────────────────┴───┴───────┴─────┴───┴─────┘

The function `V` takes a vector and returns a multivector. When used dyadically, the left argument specifies the components of the multivector to fill with the values given as right argument, adding repeated elements. Multivectors are created by `V` in two steps. First, a vector of zeros is defined to fit all the given components. Then, that vector is filled adding the elements given as right argument.

In [30]:
V ← {⍺←2*⍳≢⍵ ⋄ r←⍺⌊⍥⍴⍵ ⋄ v←0⍴⍨1+⌈/,a←r↑⍺ ⋄ v[a]+←r↑⍵ ⋄ v}  ⍝ multi(V)ector with components ⍵ at ⍺ (default vector)
⍝ eg
'V 1 2 3 4' 'V v' '3 5 6 V v' Table (V 1+⍳4) (V v) (3 5 6 V v)
Assert 0 1 2 0 3 ≡ V 1 2 3
Assert 0 0 1 2 0 0 3 ≡ 2 3 6 V 1 2 3
Assert (+/⍳4) = (4⍴0) V ⍳4

┌─────────────────┬──────────────────────────┬──────────────────────────────┐
│V 1 2 3 4 │V v │3 5 6 V v │
├─────────────────┼──────────────────────────┼──────────────────────────────┤
│0 1 2 0 3 0 0 0 4│┌─┬──────┬──────┬─┬──────┐│┌─┬─┬─┬──────┬─┬──────┬──────┐│
│ ││0│4.991 │0.4499│0│0.176 │││0│0│0│4.991 │0│0.4499│0.176 ││
│ ││ │3.075 │4.298 │ │1.162 │││ │ │ │3.075 │ │4.298 │1.162 ││
│ ││ │2.979 │3.19 │ │0.823 │││ │ │ │2.979 │ │3.19 │0.823 ││
│ ││ │0.4763│0.2869│ │0.3809│││ │ │ │0.4763│ │0.2869│0.3809││
│ ││ │0.9872│2.65 │ │3.207 │││ │ │ │0.9872│ │2.65 │3.207 ││
│ │└─┴──────┴──────┴─┴──────┘│└─┴─┴─┴──────┴─┴──────┴──────┘│
└─────────────────┴──────────────────────────┴──────────────────────────────┘

We also define a function `GB` to get the components of the multivector grouped in blades (a *blade of grade n* is just another name for an n-vector). With a left argument, `GB` will return the blades of those grades.

In [31]:
GB ← {b←Z({⊂⊂⍵}⌸+/¨BB≢⍵)⌷¨⊂,⍵ ⋄ ⍺←⍳≢b ⋄ (⊂⍺)⌷(1+⌈/,⍺)X b}   ⍝ (G)roup in (B)lades
'multivector' 'scalar' 'vector' 'bivector' 'trivector' Table (⊂⍳8),GB⍳8
Assert (GB⍳8) ≡ (⍳4)(⊃⍤GB)¨⊂⍳8

┌───────────────┬──────┬──────┬────────┬─────────┐
│multivector │scalar│vector│bivector│trivector│
├───────────────┼──────┼──────┼────────┼─────────┤
│0 1 2 3 4 5 6 7│0 │1 2 4 │3 5 6 │7 │
└───────────────┴──────┴──────┴────────┴─────────┘

### Alternative representations

There are other representations we could have chosen for multivectors. Perhaps one of the most obvious choices would be to represent multivectors as multidimensional arrays, where the number of dimensions in the array corresponds with the number of dimensions of the multivector. This array would therefore have two components along each dimension. For example:

In [32]:
           0  ⍝ 0D: scalar
        ⍳2*1  ⍝ 1D: scalar, x
    2 2⍴⍳2*2  ⍝ 2D: scalar, x, y, yx
  2 2 2⍴⍳2*3  ⍝ 3D: scalar, x, y, yx, z, zx, zy, zyx
2 2 2 2⍴⍳2*4  ⍝ 4D: scalar, x, y, yx, z, zx, zy, zyx, t, ... , tzyx

0

0 1

0 1
2 3

0 1
2 3

4 5
6 7

0 1
 2 3

 4 5
 6 7


 8 9
10 11

12 13
14 15

A function `MD` to get a multidimensional multivector can be defined as:

In [33]:
MD ← {⍺←D⍵ ⋄ (⍺⍴2)⍴(2*⍺)X⍵}   ⍝ represent multivector as (M)ulti(D)imensional array
⍝ eg
3 MD¨0 (0 1) (V 1+⍳3) (⍳2*4)
  MD¨0 (0 1) (V 1+⍳3) (⍳2*4)
⊢im←MD id

┌───┬───┬───┬───┐
│0 0│0 1│0 1│0 1│
│0 0│0 0│2 0│2 3│
│ │ │ │ │
│0 0│0 0│3 0│4 5│
│0 0│0 0│0 0│6 7│
└───┴───┴───┴───┘

┌─┬───┬───┬─────┐
│0│0 1│0 1│ 0 1│
│ │ │2 0│ 2 3│
│ │ │ │ │
│ │ │3 0│ 4 5│
│ │ │0 0│ 6 7│
│ │ │ │ │
│ │ │ │ │
│ │ │ │ 8 9│
│ │ │ │10 11│
│ │ │ │ │
│ │ │ │12 13│
│ │ │ │14 15│
└─┴───┴───┴─────┘

┌──┬───┐
│1 │x │
├──┼───┤
│y │yx │
└──┴───┘
┌──┬───┐
│z │zx │
├──┼───┤
│zy│zyx│
└──┴───┘

Although this representation offers some nice properties, such as being indexed using the bub (when `⎕IO=0`), it also requires to always use a number of components which is a power of 2. If, for example, we want to work with 3D vectors, it will be inconvenient using arrays of 8 components. Moreover, it will in general be simpler to work with vectors than with multidimensional arrays.

Another question is how to choose the sign of our units. The simple approach choosen here (the inverse lexicographic order is always positive) lacks some nice properties of other options. In particular, there are some advantages when the units are choosen such that all the units introduced when adding a dimension are left complements of the elements of lower dimensions (ie. the result of multiplying by them by the left is the pseudoscalar). If we choose the pseudoscalars in inverse lexicographic order, the base up to 3D would be:

    0D: 1
    1D: 1 x
    2D: 1 x y yx
    3D: 1 x y yx z xz zy zyx
    
Notice that the bivector `xz` is positive (instead of `zx`). This is so because the left complement of `y`, for the pseudoscalar `zyx` is `xz` (the result of `xz ∆ y` is `zyx`). Left complements and pseudoscalars will be further discussed later.

## Geometric product of unit multivectors

We are ready to define the geometric product of multivectors. We will represent it with the symbol `∆`. First, we will look at the product of unit multivectors.

We know that the result of squaring a unitary vector is `1`, and also that the product of orthogonal vectors is anticommutative:

    1 ≡ ∆⍨V U v                      ⍝ square of unitary vector is 1
    (u +.× v ≡ 0) ∧ u (∆⍨≡-⍤∆)⍥V v   ⍝ anticommutative product of orthogonal vectors

When multiplying multivectors, we only need to apply these two rules, in addition to the associative and distributive properties. For example (we consider the 3D base defined above):

    (yx ∆ x) ≡ (y ∆ x) ∆ x    ⋄  (yx ∆ x) ≡ y ∆ (x ∆ x)      ⋄  (yx ∆ x) ≡  y ∆ 1   ⋄  (yx ∆ x) ≡  y
    (yx ∆ y) ≡ (y ∆ x) ∆ y    ⋄  (yx ∆ y) ≡ (-x ∆ y) ∆ y     ⋄  (yx ∆ y) ≡ -x ∆ 1   ⋄  (yx ∆ y) ≡ -x
    (∆⍨ yx)  ≡ y ∆ x ∆ y ∆ x  ⋄  (∆⍨ yx)  ≡ - y ∆ x ∆ x ∆ y  ⋄  (∆⍨ yx)  ≡ - y ∆ y  ⋄  (∆⍨ yx)  ≡ ¯1 

The same rules apply for more dimensions:

    (zyx ∆ yx) ≡ z ∆ yx ∆ yx        ⋄  (zyx ∆ yx) ≡ z ∆ ¯1               ⋄  (zyx ∆ yx) ≡ - z
    (zyx ∆ zy) ≡ z ∆ y ∆ x ∆ z ∆ y  ⋄  (zyx ∆ zy) ≡ - z ∆ z ∆ y ∆ x ∆ x  ⋄  (zyx ∆ zy) ≡ - y
    (zyx ∆ y)  ≡ z ∆ yx ∆ y         ⋄  (zyx ∆ y)  ≡ z ∆ -x               ⋄  (zyx ∆ y)  ≡ - zx
    (∆⍨ zyx)   ≡ z ∆ yx ∆ z ∆ yx    ⋄  (∆⍨ zyx)   ≡ yx ∆ yx              ⋄  (∆⍨zyx)    ≡ ¯1
    (zx ∆ yx)  ≡ z ∆ x ∆ y ∆ x      ⋄  (zx ∆ yx)  ≡ z ∆ -y               ⋄  (zx ∆ yx)  ≡ - zy

It is interesting to point out that the square of a bivector or a trivector is `¯1`. This means that `yx` and `zyx` are solutions to `¯1*÷2`, just like `0j1`!

### Multiplication table

In the example above, we have calculated almost all possible products between unit multivectors. Let's create a table with all of them. We are going to define a function that, given a number of components, uses the bub to find the multiplication table.

In [34]:
⍝ eg
⍝ base unitary base for 1, 2 and 3 dimensions
b1 ← BB 2*1  ⍝ 1D bub: 2*1 elements (scalar, x)
b2 ← BB 2*2  ⍝ 2D bub: 2*2 elements (scalar, x, y, yx)
b3 ← BB 2*3  ⍝ 3D bub: 2*3 elements (scalar, x, y, yx, z, zx, zy, zyx)
'1D' '2D' '3D' Table b1 b2 b3

┌─────┬─────────────────┬─────────────────────────────────────────────────┐
│1D │2D │3D │
├─────┼─────────────────┼─────────────────────────────────────────────────┤
│┌─┬─┐│┌───┬───┬───┬───┐│┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐│
││0│1│││0 0│0 1│1 0│1 1│││0 0 0│0 0 1│0 1 0│0 1 1│1 0 0│1 0 1│1 1 0│1 1 1││
│└─┴─┘│└───┴───┴───┴───┘│└─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┘│
└─────┴─────────────────┴─────────────────────────────────────────────────┘

Parallel vectors, when multiplied, result in a scalar value. Therefore, if a 1-vector is in both factors, it will not be present in the product. For example, in:

    (zyx ∆ y)  ≡ - zx  ⋄  (zyx ∆ yx) ≡ - z  ⋄  (zx ∆ yx)  ≡ - zy  ⋄  ¯1 ≡ ∆⍨ yx
    
we see that when a vector (`x`, `y` or `z`) is found in both factors, it does not appear in the result. In other words, each vector will be present in the product if it is in one and only one of the factors. Therefore, to obtain the position of the product of two bub, we first perform a XOR (`≠`), and then convert it back to index value with `2⊥`.

In [35]:
PB ← 2⊥≠   ⍝ (P)osition of product of (B)ub
⍝ eg
'∘.≠' '∘.(2⊥≠)' Table (∘.≠⍨ b3) (,¨∘.(2⊥≠)⍨ b3)  ⍝ xor table
'1D' '2D' '3D' Table ∘.PB⍨¨ b1 b2 b3             ⍝ position of products
'1D' '2D' '3D' Table (⌷∘id⊂)¨∘.PB⍨¨ b1 b2 b3     ⍝ name of products

┌─────────────────────────────────────────────────┬─────────────────┐
│∘.≠ │∘.(2⊥≠) │
├─────────────────────────────────────────────────┼─────────────────┤
│┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐│┌─┬─┬─┬─┬─┬─┬─┬─┐│
││0 0 0│0 0 1│0 1 0│0 1 1│1 0 0│1 0 1│1 1 0│1 1 1│││0│1│2│3│4│5│6│7││
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│├─┼─┼─┼─┼─┼─┼─┼─┤│
││0 0 1│0 0 0│0 1 1│0 1 0│1 0 1│1 0 0│1 1 1│1 1 0│││1│0│3│2│5│4│7│6││
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│├─┼─┼─┼─┼─┼─┼─┼─┤│
││0 1 0│0 1 1│0 0 0│0 0 1│1 1 0│1 1 1│1 0 0│1 0 1│││2│3│0│1│6│7│4│5││
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│├─┼─┼─┼─┼─┼─┼─┼─┤│
││0 1 1│0 1 0│0 0 1│0 0 0│1 1 1│1 1 0│1 0 1│1 0 0│││3│2│1│0│7│6│5│4││
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│├─┼─┼─┼─┼─┼─┼─┼─┤│
││1 0 0│1 0 1│1 1 0│1 1 1│0 0 0│0 0 1│0 1 0│0 1 1│││4│5│6│7│0│1│2│3││
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│├─┼─┼─┼─┼─┼─┼─┼─┤│
││1 0 1│1 0 0│1 1 1│1 1 0│0 0 1│0 0 0│0 1 1│0 1 0│││5│4│7│6│1│0│3│2││
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│├─┼─┼─┼─┼─┼─┼─┼─┤│
││1 1 0│1 1 1│1 0 0│1 0 1│0 1 0│0 1 1│0 0 0│0 0 1│││6│7│4│5│2│3│0│1││
│├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤│├─┼─┼─┼─┼─┼─┼─┼─┤│
││1 1 1│1 1 0│1 0 1│1 0 0│0 1 1│0 1 0│0 0 1│0 0 0│││7│6│5│4│3│2│1│0││
│└─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┘│└─┴─┴─┴─┴─┴─┴─┴─┘│
└─────────────────────────────────────────────────┴─────────────────┘

┌───┬───────┬───────────────┐
│1D │2D │3D │
├───┼───────┼───────────────┤
│0 1│0 1 2 3│0 1 2 3 4 5 6 7│
│1 0│1 0 3 2│1 0 3 2 5 4 7 6│
│ │2 3 0 1│2 3 0 1 6 7 4 5│
│ │3 2 1 0│3 2 1 0 7 6 5 4│
│ │ │4 5 6 7 0 1 2 3│
│ │ │5 4 7 6 1 0 3 2│
│ │ │6 7 4 5 2 3 0 1│
│ │ │7 6 5 4 3 2 1 0│
└───┴───────┴───────────────┘

┌─────┬─────────────┬─────────────────────────────────┐
│1D │2D │3D │
├─────┼─────────────┼─────────────────────────────────┤
│┌─┬─┐│┌──┬──┬──┬──┐│┌───┬───┬───┬───┬───┬───┬───┬───┐│
││1│x│││1 │x │y │yx│││1 │x │y │yx │z │zx │zy │zyx││
│├─┼─┤│├──┼──┼──┼──┤│├───┼───┼───┼───┼───┼───┼───┼───┤│
││x│1│││x │1 │yx│y │││x │1 │yx │y │zx │z │zyx│zy ││
│└─┴─┘│├──┼──┼──┼──┤│├───┼───┼───┼───┼───┼───┼───┼───┤│
│ ││y │yx│1 │x │││y │yx │1 │x │zy │zyx│z │zx ││
│ │├──┼──┼──┼──┤│├───┼───┼───┼───┼───┼───┼───┼───┤│
│ ││yx│y │x │1 │││yx │y │x │1 │zyx│zy │zx │z ││
│ │└──┴──┴──┴──┘│├───┼───┼───┼───┼───┼───┼───┼───┤│
│ │ ││z │zx │zy │zyx│1 │x │y │yx ││
│ │ │├───┼───┼───┼───┼───┼───┼───┼───┤│
│ │ ││zx │z │zyx│zy │x │1 │yx │y ││
│ │ │├───┼───┼───┼───┼───┼───┼───┼───┤│
│ │ ││zy │zyx│z │zx │y │yx │1 │x ││
│ │ │├───┼───┼───┼───┼───┼───┼───┼───┤│
│ │ ││zyx│zy │zx │z │yx │y │x │1 ││
│ │ │└───┴───┴───┴───┴───┴───┴───┴───┘│
└─────┴─────────────┴─────────────────────────────────┘

The n-dimensional table is just a subset of the (n+1)-dimensional table. We also see that the product is closed for a number of dimensions, but not for a number of components.

The multiplication table is not finished yet. We still need to determine the sign resulting from the operation. As we discussed before, by convention, we will assume that a product is positive when vectors of higher dimensions multiply vectors of lower dimensions from left to right (ie. `0 0 0 1` is `1 0 ∆V 0 1`; the result of `0 1 ∆V 1 0` will be `0 0 0 ¯1`). And remember that we will multiply by `¯1` each time we swap factors, because the product of orthonormal vectors is anticommutative. So, we need to count the number of vectors that are multiplying, by the right, vectors of higher dimensions, and multiply by `¯1` for each of them. Notice that we ignore the first component of the left argument (`1↓⊣`) and the last component of the right one (`¯1↓⊢`).

In [36]:
SB ← ¯1*(+\¯1↓⊢)+.×1↓⊣   ⍝ (S)ign of product of (B)ub
⍝ eg
l ← 0 1 1 0 1  ⋄  ln ← l/'utzyx'  ⋄  ll ← 1↓l                   ⍝ left argument
r ← 1 0 1 1 0  ⋄  rn ← r/'utzyx'  ⋄  rr ← +\¯1↓r                ⍝ right argument
s ← ' tzxuzy' '-txzuzy' ' txuy' '-tuxy' ' utxy' '-utyx'         ⍝ sequence
p ← ' ' 'zx ≡ -xz' 'uz ≡ -zu' 'xu ≡ -ux' 'tu ≡ -ut' 'xy ≡ -yx'  ⍝ anticommutative products
np ← ⍳6                                                         ⍝ number of permutations
(ln,' ∆ ',rn) 'p' 'np' '¯1*np' Table∘(⍪¨) s p (,¨np) (,¨¯1*np)
h ← ('l←',ln) ('r←',rn) 'll←1↓l' 'rr←+\¯1↓r' 'll×rr' 'll+.×rr' '¯1*ll+.×rr'
h Table l r ll rr (ll×rr) (ll+.×rr) (¯1*ll+.×rr)
'1D' '2D' '3D' Table ∘.SB⍨¨ b1 b2 b3

┌─────────┬──────────┬───┬─────┐
│tzx ∆ uzy│p │np │¯1*np│
├─────────┼──────────┼───┼─────┤
│┌───────┐│┌────────┐│┌─┐│┌──┐ │
││ tzxuzy│││ │││0│││1 │ │
│├───────┤│├────────┤│├─┤│├──┤ │
││-txzuzy│││zx ≡ -xz│││1│││¯1│ │
│├───────┤│├────────┤│├─┤│├──┤ │
││ txuy │││uz ≡ -zu│││2│││1 │ │
│├───────┤│├────────┤│├─┤│├──┤ │
││-tuxy │││xu ≡ -ux│││3│││¯1│ │
│├───────┤│├────────┤│├─┤│├──┤ │
││ utxy │││tu ≡ -ut│││4│││1 │ │
│├───────┤│├────────┤│├─┤│├──┤ │
││-utyx │││xy ≡ -yx│││5│││¯1│ │
│└───────┘│└────────┘│└─┘│└──┘ │
└─────────┴──────────┴───┴─────┘

┌─────────┬─────────┬───────┬─────────┬───────┬───────┬──────────┐
│l←tzx │r←uzy │ll←1↓l │rr←+\¯1↓r│ll×rr │ll+.×rr│¯1*ll+.×rr│
├─────────┼─────────┼───────┼─────────┼───────┼───────┼──────────┤
│0 1 1 0 1│1 0 1 1 0│1 1 0 1│1 1 2 3 │1 1 0 3│5 │¯1 │
└─────────┴─────────┴───────┴─────────┴───────┴───────┴──────────┘

┌───┬─────────┬─────────────────────┐
│1D │2D │3D │
├───┼─────────┼─────────────────────┤
│1 1│1 1 1 1│1 1 1 1 1 1 1 1│
│1 1│1 1 ¯1 ¯1│1 1 ¯1 ¯1 ¯1 ¯1 1 1│
│ │1 1 1 1│1 1 1 1 ¯1 ¯1 ¯1 ¯1│
│ │1 1 ¯1 ¯1│1 1 ¯1 ¯1 1 1 ¯1 ¯1│
│ │ │1 1 1 1 1 1 1 1│
│ │ │1 1 ¯1 ¯1 ¯1 ¯1 1 1│
│ │ │1 1 1 1 ¯1 ¯1 ¯1 ¯1│
│ │ │1 1 ¯1 ¯1 1 1 ¯1 ¯1│
└───┴─────────┴─────────────────────┘

In fact, we do not need to know the total number of permutations, only its parity. So, instead of adding them, we can use the `≠` function again (which, remember, is XOR for booleans):

In [37]:
SB ← ¯1*(≠\¯1↓⊢)≠.∧1↓⊣   ⍝ (S)ign of product of (B)ub
⍝ eg
Assert (∘.(¯1*(+\¯1↓⊢)+.×1↓⊣)⍨¨ ≡ ∘.SB⍨¨) b1 b2 b3

This is the function to calculate the multiplication table:

In [38]:
 BB  ← ↓∘⍉2⊥⍣¯1⍳           ⍝ (B)inary unitary (B)ase for multivectors of ⍵ components
 PB  ← 2⊥≠                 ⍝ (P)osition of product of (B)inary unitary base
 SB  ← ¯1*(≠\¯1↓⊢)≠.∧1↓⊣   ⍝ (S)ign of product of (B)inary unitary base
 PS  ← (∘.PB,⍥⊂∘.SB)⍨∘BB   ⍝ (P)osition and (S)ign multiplication tables for multivectors of ⍵ components
 TD  ← PS 2∘*              ⍝ multiplication (T)ables for multivectors of ⍵ (D)imensions
 BN  ← ⊃((-1+⊣)↑¨⊢)/⍤PS    ⍝ (B)ase multiplication table for multivectors of ⍵ compo(N)ents
⊢BD  ← ⊃((-1+⊣)↑¨⊢)/⍤TD    ⍝ (B)ase multiplication table for multivectors of ⍵ (D)imensions
⍝ eg
'1D' '2D' Table BD¨1 2   ⍝ 1D and 2D tables
(⊂'3D')Table,⊂TD 3       ⍝ 3D positions and signs tables

┌┴─┐
 ⊢ ┌┴┐
 ⊃ ⍤
 ┌───────┴───────┐
 / ┌────┴────┐
 ┌─┘ ∘ ∘
 ┌───────┼─┐ ┌───┴───┐ ┌┴┐
┌─┴─┐ ¨ ⊢ ⍨ ┌─┴──┐ 2 *
- ┌─┼─┐ ┌─┘ ┌─┘ ∘ ┌─┼─┐
 1 + ⊣ ↑ ┌──────┼────┐ ┌┴┐ 2 ⍣ ⍳
 . ⍥ . ↓ ⍉ ┌┴┐
 ┌─┴─┐ ┌┴┐ ┌─┴──┐ ⊥ ¯1
 ∘ ┌─┼─┐ , ⊂ ∘ ┌──┼─────────┐
 2 ⊥ ≠ ¯1 * ┌───────┼────┐
 ┌─┴──┐ . ┌─┼─┐
 \ ┌──┼─┐ ┌┴┐ 1 ↓ ⊣
 ┌─┘ ¯1 ↓ ⊢ ≠ ∧
 ≠

┌─────────┬──────────────────────────────────┐
│1D │2D │
├─────────┼──────────────────────────────────┤
│┌───┬───┐│┌───────┬───────┬────────┬───────┐│
││1 │0 1│││1 │0 1 │0 0 1 │0 0 0 1││
│├───┼───┤│├───────┼───────┼────────┼───────┤│
││0 1│1 │││0 1 │1 │0 0 0 ¯1│0 0 ¯1 ││
│└───┴───┘│├───────┼───────┼────────┼───────┤│
│ ││0 0 1 │0 0 0 1│1 │0 1 ││
│ │├───────┼───────┼────────┼───────┤│
│ ││0 0 0 1│0 0 1 │0 ¯1 │¯1 ││
│ │└───────┴───────┴────────┴───────┘│
└─────────┴──────────────────────────────────┘

┌───────────────────────────────────────┐
│3D │
├───────────────────────────────────────┤
│┌───────────────┬─────────────────────┐│
││0 1 2 3 4 5 6 7│1 1 1 1 1 1 1 1││
││1 0 3 2 5 4 7 6│1 1 ¯1 ¯1 ¯1 ¯1 1 1││
││2 3 0 1 6 7 4 5│1 1 1 1 ¯1 ¯1 ¯1 ¯1││
││3 2 1 0 7 6 5 4│1 1 ¯1 ¯1 1 1 ¯1 ¯1││
││4 5 6 7 0 1 2 3│1 1 1 1 1 1 1 1││
││5 4 7 6 1 0 3 2│1 1 ¯1 ¯1 ¯1 ¯1 1 1││
││6 7 4 5 2 3 0 1│1 1 1 1 ¯1 ¯1 ¯1 ¯1││
││7 6 5 4 3 2 1 0│1 1 ¯1 ¯1 1 1 ¯1 ¯1││
│└───────────────┴─────────────────────┘│
└───────────────────────────────────────┘

## Geometric product of multivectors

In general, if we have two multivectors, we will apply the distributive property, so we need to calculate the outer product of its components, then multiply by the multiplication table, and add all the results. Let's see an example:

In [39]:
⍝ eg
'uv' Table u1 v1 ← ⊃¨¨u v                                               ⍝ 2 multivectors of 3 components: 1, x, y
t ← u1(BN⌈⍥≢)v1                                                         ⍝ multiplication table
't' 'u∘.×v' 'u(t×∘.×)v'       Table t (,¨u1∘.×v1) (u1(t×∘.×)v1)         ⍝ products
'↑,u(t×∘.×)v' '+⌿↑,u(t×∘.×)v' Table (↑,u1(t×∘.×)v1) (+⌿↑,u1(t×∘.×)v1)   ⍝ mix and reduce

┌─────────────┬──────────────────┐
│u │v │
├─────────────┼──────────────────┤
│1.017 0 1.343│4.991 0.4499 0.176│
└─────────────┴──────────────────┘

┌────────────────────────┬─────────────────────┬──────────────────────────────────┐
│t │u∘.×v │u(t×∘.×)v │
├────────────────────────┼─────────────────────┼──────────────────────────────────┤
│┌─────┬───────┬────────┐│┌─────┬──────┬──────┐│┌─────────┬────────────┬─────────┐│
││1 │0 1 │0 0 1 │││5.077│0.4577│0.179 │││5.077 │0 0.4577 │0 0 0.179││
│├─────┼───────┼────────┤│├─────┼──────┼──────┤│├─────────┼────────────┼─────────┤│
││0 1 │1 │0 0 0 ¯1│││0 │0 │0 │││0 0 │0 │0 0 0 0 ││
│├─────┼───────┼────────┤│├─────┼──────┼──────┤│├─────────┼────────────┼─────────┤│
││0 0 1│0 0 0 1│1 │││6.705│0.6044│0.2364│││0 0 6.705│0 0 0 0.6044│0.2364 ││
│└─────┴───────┴────────┘│└─────┴──────┴──────┘│└─────────┴────────────┴─────────┘│
└────────────────────────┴─────────────────────┴──────────────────────────────────┘

┌──────────────────────────┬─────────────────────────┐
│↑,u(t×∘.×)v │+⌿↑,u(t×∘.×)v │
├──────────────────────────┼─────────────────────────┤
│5.077 0 0 0 │5.314 0.4577 6.884 0.6044│
│0 0.4577 0 0 │ │
│0 0 0.179 0 │ │
│0 0 0 0 │ │
│0 0 0 0 │ │
│0 0 0 0 │ │
│0 0 6.705 0 │ │
│0 0 0 0.6044│ │
│0.2364 0 0 0 │ │
└──────────────────────────┴─────────────────────────┘

A function to calculate the geometric product of two multivectors can therefore be defined as:

In [40]:
GP ← {t ← ⍺(BN⌈⍥≢)⍵ ⋄ +⌿↑,⍺(t×(⍴t)⊂¨⍤↑∘.×)⍵}   ⍝ geometric product
⍝ or as a tacit expression:
⊢ P ← +⌿∘↑∘,∘.×(⊂¨⍤↑⍨∘⍴×⊢)BN⍤⌈⍥≢
⍝ eg
'u' 'v' 'u P⍥V v' Table u v (u P⍥V v)
Assert (u P v) ≡ u GP v

┌──┴───┐
 ⊢ ┌────┴────┐
 ∘ ┌─────┼──────────┐
 ┌┴┐ . ┌─┼─┐ ⍥
 ∘ , ┌┴┐ ∘ × ⊢ ┌┴┐
 ┌┴┐ ∘ × ┌┴┐ ⍤ ≢
 ⌿ ↑ ⍨ ⍴ ┌┴─┐
┌─┘ ┌─┘ ┌┴┐ ⌈
+ ⍤ ⊃ ⍤
 ┌┴┐ ┌─────┴──────┐
 ¨ ↑ / ∘
 ┌─┘ ┌─┘ ┌───┴───┐
 ⊂ ┌───────┼─┐ ⍨ ┌─┴──┐
 ┌─┴─┐ ¨ ⊢ ┌─┘ ∘ ┌─┼─┐
 - ┌─┼─┐ ┌─┘ ┌──────┼────┐ ┌┴┐ 2 ⍣ ⍳
 1 + ⊣ ↑ . ⍥ . ↓ ⍉ ┌┴┐
 ┌─┴─┐ ┌┴┐ ┌─┴──┐ ⊥ ¯1
 ∘ ┌─┼─┐ , ⊂ ∘ ┌──┼─────────┐
 2 ⊥ ≠ ¯1 * ┌───────┼────┐
 ┌─┴──┐ . ┌─┼─┐
 \ ┌──┼─┐ ┌┴┐ 1 ↓ ⊣
 ┌─┘ ¯1 ↓ ⊢ ≠ ∧
 ≠

┌───────────────────┬──────────────────────┬──────────────────────────────────────────┐
│u │v │u P⍥V v │
├───────────────────┼──────────────────────┼──────────────────────────────────────────┤
│┌───────┬─┬───────┐│┌──────┬──────┬──────┐│┌──────┬─┬─┬────────┬─┬───────┬────────┬─┐│
││1.017 │0│1.343 │││4.991 │0.4499│0.176 │││ 5.314│0│0│ ¯0.4577│0│ 6.526 │ 0.6044 │0││
││4.983 │ │2.48 │││3.075 │4.298 │1.162 │││18.21 │0│0│¯21.42 │0│ 1.837 │10.66 │0││
││1.231 │ │3.193 │││2.979 │3.19 │0.823 │││ 6.296│0│0│ ¯3.927 │0│ 8.499 │10.18 │0││
││2.557 │ │0.08875│││0.4763│0.2869│0.3809│││ 1.252│0│0│ ¯0.7338│0│¯0.9319│ 0.02546│0││
││0.04726│ │4.971 │││0.9872│2.65 │3.207 │││15.99 │0│0│ ¯0.1252│0│ 4.755 │13.17 │0││
│└───────┴─┴───────┘│└──────┴──────┴──────┘│└──────┴─┴─┴────────┴─┴───────┴────────┴─┘│
└───────────────────┴──────────────────────┴──────────────────────────────────────────┘

Notice that we enclose the elements of the result of the outer product to allow the multiplication of nested multivectors. We are also going to define an operator `_GP_` that takes as left operand the multiplication function, and as right operand the multiplication table or a function to calculate it:

In [41]:
_GP_ ← {t ← ⍺(⍵⍵⍨⍨)⍵ ⋄ +⌿↑,⍺(t×∘⊂¨(⍴t)↑∘.(⍺⍺_Z))⍵}   ⍝ geometric product operator
_GP  ← _GP_(BN⌈⍥≢)                                   ⍝ geometric product operator (will compute table)
_GP2 ← _GP_(BD 2)                                    ⍝ geometric product 2D (will use precomputed table)
 GP  ← ×_GP  _V                                      ⍝ geometric product function (will compute table)
 GP2 ← ×_GP2 _V                                      ⍝ geometric product 2D (will use precomputed table)
⍝ eg
v1 ← (1 2) (3 4) ⋄ v2 ← (5 6) (7 8)   ⍝ 2D vectors
'v1' 'v2' '×_GP' '∘.×_GP' '+.×_GP' Table v1 v2 (v1(×_GP)⍥V v2) (v1(∘.×_GP)⍥V v2) (v1(+.×_GP)⍥V v2)
]RunTime 'v1   ×_GP v2' 'v1   ×_GP2 v2' -compare
]RunTime 'v1 ∘.×_GP v2' 'v1 ∘.×_GP2 v2' -compare

┌─────────┬─────────┬───────────────────┬────────────────────┬─────────┐
│v1 │v2 │×_GP │∘.×_GP │+.×_GP │
├─────────┼─────────┼───────────────────┼────────────────────┼─────────┤
│┌───┬───┐│┌───┬───┐│┌─────┬───┬───┬───┐│┌─────┬───┬───┬────┐│70 0 0 16│
││1 2│3 4│││5 6│7 8│││26 44│0 0│0 0│8 8│││26 30│0 0│0 0│8 10││ │
│└───┴───┘│└───┴───┘│└─────┴───┴───┴───┘││38 44│0 0│0 0│6 8││ │
│ │ │ │└─────┴───┴───┴────┘│ │
└─────────┴─────────┴───────────────────┴────────────────────┴─────────┘

v1 ×_GP v2 → 3.9E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* v1 ×_GP2 v2 → 3.8E¯5 | -3% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

v1 ∘.×_GP v2 → 8.3E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* v1 ∘.×_GP2 v2 → 9.0E¯5 | +8% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

Using a precomputed table suposes only a small improvement. We will work on the performance of the geometric product later.

## Properties of the geometric product

As it has been defined, the result of multiplying a vector by its inverse using the geometric product (`GP∘I⍨`) must result in the scalar `1`. Indeed:

In [42]:
Assert 1 E¨ (GP∘I⍨)∘V¨ u v w

This is not the only property that a geometric product must fulfill. We have already used several other properties such as associativity or the distributive property. Let's clearly define the properties of the geometric product:

In [43]:
]dinput
_GA ← {                                               ⍝ (G)eometric (A)lgebra with geometric product ⍺⍺
    _ ← Assert (a b ← ⍺)        VS (u v w ← ⍵)        ⍝   vector space: a b scalars, u v w vectors
    _ ← Assert (u ⍺⍺ v ⍺⍺ w)    E w ⍺⍺⍨ u ⍺⍺ v        ⍝   associative product
    _ ← Assert (u∘⍺⍺ ⊃+_V/ v w) E (⊃+_V/ u∘⍺⍺¨ v w)   ⍝   distributive by left
    _ ← Assert (⍺⍺∘u ⊃+_V/ v w) E (⊃+_V/ ⍺⍺∘u¨ v w)   ⍝   distributive by right
    _ ← Assert (a ⍺⍺ v ⍺⍺ b)    E v × a × b           ⍝   extension of scalar product
        Assert (1 E ⍺⍺⍨)¨       V¨BV⌈/D¨⍵             ⍝   unitary vectors square to 1
}

When we have a vector space with a product that follows these properties, we say we have a *geometric algebra*. The operator `_GA` is analogous to `VS`, but takes as operand a geometric product function. If we try the functions we already have:

In [44]:
a b (GP  _GA)V¨ u v w   ⍝ geometric algebra with geometric product ∆ (u v w are vectors)
a b (GP2 _GA)   u v w   ⍝ geometric algebra with geometric product ∆2 (u v w are multivectors)

The `GP` and `GP2` functions are geometric products.

## Improving the performance of the geometric product

The performance of our geometric product can be improved. We can do much better using the `V` function defined before instead of mix and reduce. As we have defined it, `V` can be used to build a multivector from its components. For example, we can use it to calculate the base multivectors product table from the tables of positions and signs:

In [45]:
BT ← ⊃V¨/     ⍝ (B)ase multivectors from position and sign (T)ables
BN  ← BT PS   ⍝ (B)ase multivectors multiplication table for ⍵ compo(N)ents
BD  ← BT TD   ⍝ (B)ase multivectors multiplication table for ⍵ (D)imensions
⍝ eg
'1D' '2D' Table BD¨1 2   ⍝ same result as before

┌─────────┬──────────────────────────────────┐
│1D │2D │
├─────────┼──────────────────────────────────┤
│┌───┬───┐│┌───────┬───────┬────────┬───────┐│
││1 │0 1│││1 │0 1 │0 0 1 │0 0 0 1││
│├───┼───┤│├───────┼───────┼────────┼───────┤│
││0 1│1 │││0 1 │1 │0 0 0 ¯1│0 0 ¯1 ││
│└───┴───┘│├───────┼───────┼────────┼───────┤│
│ ││0 0 1 │0 0 0 1│1 │0 1 ││
│ │├───────┼───────┼────────┼───────┤│
│ ││0 0 0 1│0 0 1 │0 ¯1 │¯1 ││
│ │└───────┴───────┴────────┴───────┘│
└─────────┴──────────────────────────────────┘

And we can also use it in the definition of the geometric product. This is our new version:

In [46]:
⍝ GEOMETRIC PRODUCT
_∆_ ← {p s←⍺(⍵⍵⍨⍨)⍵ ⋄ (r↑p)V(r↑s)×(r←(≢¨⍺⍵)⌊⍴p)↑⍺∘.(⍺⍺_Z)⍥,⍵}   ⍝ ⍺⍺ product, ⍵⍵ tables or tables func
_∆  ← _∆_(PS⌈⍥≢)                                                ⍝ ⍺⍺ product (table will be computed)
 ∆  ← ×_∆                                                       ⍝ geometric product
 ∆2 ← ×_∆_(TD 2)                                                ⍝ 2D geometric product
⍝ eg
'v1' 'v2' '×_∆⍥V' '∘.×_∆⍥V' '+.×_∆⍥V' Table v1 v2 (v1 ×_∆⍥V v2) (v1 ∘.×_∆⍥V v2) (v1 +.×_∆⍥V v2)
Assert a b (∆  _GA)V¨ u v w   ⍝ ∆ is a geometric product
Assert a b (∆2 _GA)   u v w   ⍝ ∆2 is a geometric product
Assert (u(∆ E ∆2)v)
'u' 'v' 'u ∆ v'  Table (V u) (V v) (u ∆⍥V v)

┌─────────┬─────────┬───────────────┬────────────────┬─────────┐
│v1 │v2 │×_∆⍥V │∘.×_∆⍥V │+.×_∆⍥V │
├─────────┼─────────┼───────────────┼────────────────┼─────────┤
│┌───┬───┐│┌───┬───┐│┌─────┬─┬─┬───┐│┌─────┬─┬─┬────┐│70 0 0 16│
││1 2│3 4│││5 6│7 8│││26 44│0│0│8 8│││26 30│0│0│8 10││ │
│└───┴───┘│└───┴───┘│└─────┴─┴─┴───┘││38 44│ │ │6 8││ │
│ │ │ │└─────┴─┴─┴────┘│ │
└─────────┴─────────┴───────────────┴────────────────┴─────────┘

┌───────────────────────┬──────────────────────────┬──────────────────────────────────────────┐
│u │v │u ∆ v │
├───────────────────────┼──────────────────────────┼──────────────────────────────────────────┤
│┌─┬───────┬─┬─┬───────┐│┌─┬──────┬──────┬─┬──────┐│┌──────┬─┬─┬────────┬─┬───────┬────────┬─┐│
││0│1.017 │0│0│1.343 │││0│4.991 │0.4499│0│0.176 │││ 5.314│0│0│ ¯0.4577│0│ 6.526 │ 0.6044 │0││
││ │4.983 │ │ │2.48 │││ │3.075 │4.298 │ │1.162 │││18.21 │ │ │¯21.42 │ │ 1.837 │10.66 │ ││
││ │1.231 │ │ │3.193 │││ │2.979 │3.19 │ │0.823 │││ 6.296│ │ │ ¯3.927 │ │ 8.499 │10.18 │ ││
││ │2.557 │ │ │0.08875│││ │0.4763│0.2869│ │0.3809│││ 1.252│ │ │ ¯0.7338│ │¯0.9319│ 0.02546│ ││
││ │0.04726│ │ │4.971 │││ │0.9872│2.65 │ │3.207 │││15.99 │ │ │ ¯0.1252│ │ 4.755 │13.17 │ ││
│└─┴───────┴─┴─┴───────┘│└─┴──────┴──────┴─┴──────┘│└──────┴─┴─┴────────┴─┴───────┴────────┴─┘│
└───────────────────────┴──────────────────────────┴──────────────────────────────────────────┘

We have already checked (with the help of `_GA`) that `∆` is a geometric product. Let's check the performance and memory requirements:

In [47]:
us vs ← V¨RC∘1e4¨3 4   ⍝ large vectors (3 and 4 components of 10k rows each)
GP3D ← ×_GP_(BD 3)
GP2D ← ×_GP_(BD 2)
∆3 ← ×_∆_(TD 3)
]RunTime     'u  ∘.×_∆⍥V  v' 'u  ∘.×_GP⍥V v' -compare
]RunTime     'u  ∆3⍥V   v' 'u  GP3D⍥V v' -compare
]RunTime     'us ∆2   vs' 'us GP2D vs' -compare
]SpaceNeeded 'u  ∆3⍥V   v' 'u  GP3D⍥V v'
]SpaceNeeded 'us ∆2   vs' 'us GP2D vs'

u ∘.×_∆⍥V v → 1.4E¯4 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u ∘.×_GP⍥V v → 1.8E¯4 | +30% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

u ∆3⍥V v → 7.5E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u GP3D⍥V v → 1.3E¯4 | +72% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

us ∆2 vs → 2.6E¯4 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* us GP2D vs → 5.2E¯4 | +96% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌───────────┬────┐
│u ∆3⍥V v│2868│
├───────────┼────┤
│u GP3D⍥V v│9140│
└───────────┴────┘

┌──────────┬──────┐
│us ∆2 vs│961212│
├──────────┼──────┤
│us GP2D vs│961612│
└──────────┴──────┘

That's faster, and it needs less space. Of course, when a precomputed multiplication table is not used, it takes a bit longer (and more space), but much less than using `GP`:

In [48]:
]RunTime     'u  ∆3⍥V v ' 'u  ∆ ⍥V v ' 'u  GP⍥V v ' -compare
]RunTime     'us ∆2   vs' 'us ∆    vs' 'us GP   vs' -compare
]SpaceNeeded 'u  ∆3⍥V v ' 'u  ∆ ⍥V v ' 'u  GP⍥V v '
]SpaceNeeded 'us ∆2   vs' 'us ∆    vs' 'us GP   vs'

u ∆3⍥V v → 5.8E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 u ∆ ⍥V v → 1.2E¯4 | +104% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u GP⍥V v → 2.1E¯4 | +255% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

us ∆2 vs → 2.1E¯4 | 0% 
* us ∆ vs → 5.2E¯4 | +148% ⎕ 
* us GP vs → 2.3E¯2 | +10864% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌──────────┬────┐
│u ∆3⍥V v │2868│
├──────────┼────┤
│u ∆ ⍥V v │2940│
├──────────┼────┤
│u GP⍥V v │6492│
└──────────┴────┘

┌──────────┬───────┐
│us ∆2 vs│961212 │
├──────────┼───────┤
│us ∆ vs│1442908│
├──────────┼───────┤
│us GP vs│1639444│
└──────────┴───────┘

Finally, we can achieve another small improvement if we get rid of `_Z`, at the expense of using more memory when there are scalar zero components.

In [49]:
_∆_ ← {p s←⍺(⍵⍵⍨⍨)⍵ ⋄ (r↑p)V(r↑s)×(r←(≢¨⍺⍵)⌊⍴p)↑⍺∘.⍺⍺⍥,⍵}   ⍝ ⍺⍺ product, ⍵⍵ tables or tables func
_∆  ← _∆_(PS⌈⍥≢)                                            ⍝ ⍺⍺ product (table will be computed)
 ∆  ← ×   _∆                                                ⍝ geometric product
 ∆Z ← ×_Z _∆                                                ⍝ geometric product
⍝ eg
∆3 ← ×_∆_(TD 3)
∆3Z ← ×_Z _∆_(TD 3)
]RunTime     'u ∆3 ⍥V v' 'u ∆3Z⍥V v' -compare
]SpaceNeeded 'u ∆3 ⍥V v' 'u ∆3Z⍥V v'

u ∆3 ⍥V v → 5.8E¯5 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
* u ∆3Z⍥V v → 8.0E¯5 | +37% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌─────────┬────┐
│u ∆3 ⍥V v│3508│
├─────────┼────┤
│u ∆3Z⍥V v│2916│
└─────────┴────┘

## Products of vectors and blades

We are going to define some additional functions to make it easier to multiply vectors, just for convenience:

In [50]:
∆V ← ∆⍥V   ⍝ geometric product of vectors
∆v ← ∆∘V   ⍝ geometric product of multivector and vector
⍝ eg: 3D unitary blades: scalar, vectors, bivectors, trivector
x y z        ← 1 (0 1) (0 0 1)                   ⍝ unitary vectors
yx zx zy zyx ← (y z z ∆V¨ x x y),⊂z ∆v⍨ y ∆V x   ⍝ unitary bivectors and trivector
' ' 'x' 'y' 'z' 'yx' 'zx' 'zy' 'zyx' Table 1,(V¨x y z),yx zx zy zyx

┌─┬───┬─────┬─────────┬───────┬───────────┬─────────────┬───────────────┐
│ │x │y │z │yx │zx │zy │zyx │
├─┼───┼─────┼─────────┼───────┼───────────┼─────────────┼───────────────┤
│1│0 1│0 0 1│0 0 0 0 1│0 0 0 1│0 0 0 0 0 1│0 0 0 0 0 0 1│0 0 0 0 0 0 0 1│
└─┴───┴─────┴─────────┴───────┴───────────┴─────────────┴───────────────┘

We can use these new functions to study what is the result of multiplying base multivectors. For instance, we can observe that the product of perpendicular vectors or penpendicular bivectors is anticommutative, but the product of bivectors and orthogonal vectors is commutative:

In [51]:
⍝ anticommutative product of vector and orthogonal vector
Assert (z ∆V y) E - y ∆V z
Assert (y ∆V x) E - x ∆V y
Assert (x ∆V z) E - z ∆V x
⍝ anticommutative product of bivector and orthogonal bivector
Assert (yx ∆ zy) E - zy ∆ yx
Assert (zy ∆ zx) E - zx ∆ zy
Assert (zx ∆ yx) E - yx ∆ zx
⍝ commutative product of bivector and orthogonal vector
Assert ((V z) ∆ y ∆V x) ≡ z ∆v⍨ y ∆V x
Assert ((V y) ∆ z ∆V x) ≡ y ∆v⍨ z ∆V x
Assert ((V x) ∆ z ∆V y) ≡ x ∆v⍨ z ∆V y

# Signed geometric algebras

## Non-proper vectors

Until now, all the vectors that we have considered had the property `(∆⍨ v) ≡ +.×⍨ v`. Consequently, all vectors, when squared, resulted in a positive number (and unitary vectors, in particular, square to `1`). Well, not really.

### Vectors of negative metric

Let's consider a 3D space and calculate the square value of each of the unit multivectors.

In [52]:
gr ← G¨bu                ⍝ grades
sq ← ∆⍨¨bu               ⍝ squares
iv ← (a b VS u v,⊂)¨bu   ⍝ is a vector?
'unitary multivector' 'bub' 'name' 'grade' '∆⍨' 'vector?' Table∘(⍪¨) bu bub id (,¨gr) (,∘Z¨sq) (,¨iv)

┌───────────────────┬───────┬─────┬─────┬────┬───────┐
│unitary multivector│bub │name │grade│∆⍨ │vector?│
├───────────────────┼───────┼─────┼─────┼────┼───────┤
│┌───────────────┐ │┌─────┐│┌───┐│┌─┐ │┌──┐│┌─┐ │
││1 │ ││0 0 0│││1 │││0│ ││1 │││1│ │
│├───────────────┤ │├─────┤│├───┤│├─┤ │├──┤│├─┤ │
││0 1 │ ││0 0 1│││x │││1│ ││1 │││1│ │
│├───────────────┤ │├─────┤│├───┤│├─┤ │├──┤│├─┤ │
││0 0 1 │ ││0 1 0│││y │││1│ ││1 │││1│ │
│├───────────────┤ │├─────┤│├───┤│├─┤ │├──┤│├─┤ │
││0 0 0 1 │ ││0 1 1│││yx │││2│ ││¯1│││1│ │
│├───────────────┤ │├─────┤│├───┤│├─┤ │├──┤│├─┤ │
││0 0 0 0 1 │ ││1 0 0│││z │││1│ ││1 │││1│ │
│├───────────────┤ │├─────┤│├───┤│├─┤ │├──┤│├─┤ │
││0 0 0 0 0 1 │ ││1 0 1│││zx │││2│ ││¯1│││1│ │
│├───────────────┤ │├─────┤│├───┤│├─┤ │├──┤│├─┤ │
││0 0 0 0 0 0 1 │ ││1 1 0│││zy │││2│ ││¯1│││1│ │
│├───────────────┤ │├─────┤│├───┤│├─┤ │├──┤│├─┤ │
││0 0 0 0 0 0 0 1│ ││1 1 1│││zyx│││3│ ││¯1│││1│ │
│└───────────────┘ │└─────┘│└───┘│└─┘ │└──┘│└─┘ │
└───────────────────┴───────┴─────┴─────┴────┴───────┘

As we saw above, some unitary multivectors (the bivectors and trivectors) have negative squares. Nevertheless, they are vectors according to our definition of vector spaces (as checked by `a b VS u v,⊂`). We observe that the square of a vector will always be a scalar value. This value will be called the *metric* of the vector. Vectors which square to `1`, or another positive number, will be called *vectors of positive metric*, and the vectors that square to `¯1`, or another negative number, will be called *vectors of negative metric*. The vectors of negative metric also fulfill the anticommutative property for the product of orthogonal vectors, as well as the other properties of the geometric product. However their metric is not `1`, but `¯1`.

In [53]:
b3 ← yx zx zy            ⍝ bivectors in 3D
c ← (∆⍨≡-⍤∆)¨∘(1∘⌽)⍨b3   ⍝ check anticommutativity
' ' '∆⍨' Table∘(⍪¨) b3 (,∘Z⍤∆¨⍨b3)

┌───────────────┬────┐
│ │∆⍨ │
├───────────────┼────┤
│┌─────────────┐│┌──┐│
││0 0 0 1 │││¯1││
│├─────────────┤│├──┤│
││0 0 0 0 0 1 │││¯1││
│├─────────────┤│├──┤│
││0 0 0 0 0 0 1│││¯1││
│└─────────────┘│└──┘│
└───────────────┴────┘

Then, we can define a vector space in which the vector base is formed by `1` and the bivectors `yx`, `zx` and `zy`, for example (this space in particular would actually be the *[quaternion](https://en.wikipedia.org/wiki/Quaternion_algebra)* space). We will use our geometric product `∆` to calculate the corresponding multiplication table, and the functions `VQ` and `QV` to change of basis between quaternions and 3D multivectors:

In [54]:
VQ ← Z∘C 1 ¯2 1 0 1 1\4∘X   ⍝ multivector from quaternion
QV ← Z∘C (⊂0 3 5 6)⌷7∘X     ⍝ quaternion from multivector
⍝ eg
'q' 'VQ q' 'v' 'QV v' Table (1+⍳4) (VQ 1+⍳4) (1+⍳7) (QV 1+⍳7)   ⍝ change of basis
bq ← 1 yx zx zy                                                 ⍝ quaternions base
'basis' '∘.∆⍨' Table (⍪bq) (∘.∆⍨bq)                             ⍝ quaternions product table
'basis' 'QV¨∘.∆⍨' Table (⍪BV 4) (tq ← QV¨∘.∆⍨VQ¨BV 4)           ⍝ same table in quaternions base

┌───────┬─────────────┬─────────────┬───────┐
│q │VQ q │v │QV v │
├───────┼─────────────┼─────────────┼───────┤
│1 2 3 4│1 0 0 2 0 3 4│1 2 3 4 5 6 7│1 4 6 7│
└───────┴─────────────┴─────────────┴───────┘

┌───────────────┬──────────────────────────────────────────────────────────────────┐
│basis │∘.∆⍨ │
├───────────────┼──────────────────────────────────────────────────────────────────┤
│┌─────────────┐│┌─────────────┬────────────────┬────────────────┬────────────────┐│
││1 │││1 │0 0 0 1 │0 0 0 0 0 1 │0 0 0 0 0 0 1 ││
│├─────────────┤│├─────────────┼────────────────┼────────────────┼────────────────┤│
││0 0 0 1 │││0 0 0 1 │¯1 0 0 0 │0 0 0 0 0 0 1 0 │0 0 0 0 0 ¯1 0 0││
│├─────────────┤│├─────────────┼────────────────┼────────────────┼────────────────┤│
││0 0 0 0 0 1 │││0 0 0 0 0 1 │0 0 0 0 0 0 ¯1 0│¯1 0 0 0 0 0 0 0│0 0 0 1 0 0 0 0 ││
│├─────────────┤│├─────────────┼────────────────┼────────────────┼────────────────┤│
││0 0 0 0 0 0 1│││0 0 0 0 0 0 1│0 0 0 0 0 1 0 0 │0 0 0 ¯1 0 0 0 0│¯1 0 0 0 0 0 0 0││
│└─────────────┘│└─────────────┴────────────────┴────────────────┴────────────────┘│
└───────────────┴──────────────────────────────────────────────────────────────────┘

┌─────────┬──────────────────────────────────┐
│basis │QV¨∘.∆⍨ │
├─────────┼──────────────────────────────────┤
│┌───────┐│┌───────┬────────┬───────┬───────┐│
││1 │││1 │0 1 │0 0 1 │0 0 0 1││
│├───────┤│├───────┼────────┼───────┼───────┤│
││0 1 │││0 1 │¯1 │0 0 0 1│0 0 ¯1 ││
│├───────┤│├───────┼────────┼───────┼───────┤│
││0 0 1 │││0 0 1 │0 0 0 ¯1│¯1 │0 1 ││
│├───────┤│├───────┼────────┼───────┼───────┤│
││0 0 0 1│││0 0 0 1│0 0 1 │0 ¯1 │¯1 ││
│└───────┘│└───────┴────────┴───────┴───────┘│
└─────────┴──────────────────────────────────┘

Having a multiplication table for basis multivectors, we can use `_GP_` to get a geometric product fuction. Or we could use the 3D multiplication under a change of basis:

In [55]:
Pq ← ×_GP_ tq _V   ⍝ quaternion product using quaternion table
∆Q ← QV GP3D⍥VQ    ⍝ 3D geometric product under change of basis
'u' 'v' 'VQ u' 'VQ v' Table u (v,5) (VQ u) (VQ v,5)
'u GP3D⍥VQ v' 'u Pq v' Table (u GP3D⍥VQ v,5) (u Pq v,5)
]RunTime 'u ∆Q v,5' 'u Pq v,5' -compare

┌───────────────────┬────────────────────────┬─────────────────────────┬──────────────────────────────┐
│u │v │VQ u │VQ v │
├───────────────────┼────────────────────────┼─────────────────────────┼──────────────────────────────┤
│┌───────┬─┬───────┐│┌──────┬──────┬──────┬─┐│┌───────┬─┬─┬─┬─┬───────┐│┌──────┬─┬─┬──────┬─┬──────┬─┐│
││1.017 │0│1.343 │││4.991 │0.4499│0.176 │5│││1.017 │0│0│0│0│1.343 │││4.991 │0│0│0.4499│0│0.176 │5││
││4.983 │ │2.48 │││3.075 │4.298 │1.162 │ │││4.983 │ │ │ │ │2.48 │││3.075 │ │ │4.298 │ │1.162 │ ││
││1.231 │ │3.193 │││2.979 │3.19 │0.823 │ │││1.231 │ │ │ │ │3.193 │││2.979 │ │ │3.19 │ │0.823 │ ││
││2.557 │ │0.08875│││0.4763│0.2869│0.3809│ │││2.557 │ │ │ │ │0.08875│││0.4763│ │ │0.2869│ │0.3809│ ││
││0.04726│ │4.971 │││0.9872│2.65 │3.207 │ │││0.04726│ │ │ │ │4.971 │││0.9872│ │ │2.65 │ │3.207 │ ││
│└───────┴─┴───────┘│└──────┴──────┴──────┴─┘│└───────┴─┴─┴─┴─┴───────┘│└──────┴─┴─┴──────┴─┴──────┴─┘│
└───────────────────┴────────────────────────┴─────────────────────────┴──────────────────────────────┘

┌───────────────────────────────────────┬───────────────────────────────┐
│u GP3D⍥VQ v │u Pq v │
├───────────────────────────────────────┼───────────────────────────────┤
│┌───────┬─┬─┬──────┬─┬──────┬───────┬─┐│┌───────┬──────┬──────┬───────┐│
││ 4.841│0│0│ 7.175│0│ 6.884│ 4.482│0│││ 4.841│ 7.175│ 6.884│ 4.482││
││ 12.44 │0│0│33.82 │0│13.42 │ 14.25 │0│││ 12.44 │33.82 │13.42 │ 14.25 ││
││ 1.041│0│0│19.89 │0│10.53 │ ¯4.029│0│││ 1.041│19.89 │10.53 │ ¯4.029││
││ 1.184│0│0│ 1.178│0│ 1.016│ 12.76 │0│││ 1.184│ 1.178│ 1.016│ 12.76 ││
││¯15.89 │0│0│24.98 │0│ 5.059│¯12.94 │0│││¯15.89 │24.98 │ 5.059│¯12.94 ││
│└───────┴─┴─┴──────┴─┴──────┴───────┴─┘│└───────┴──────┴──────┴───────┘│
└───────────────────────────────────────┴───────────────────────────────┘

u ∆Q v,5 → 5.9E¯4 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 u Pq v,5 → 9.3E¯5 | -85% ⎕⎕⎕⎕⎕⎕

The operand with a precomputed table offers better performance than the change of basis. However:

In [56]:
a b (Pq _GA) u v w   ⍝ ERROR: unit vectors must square to 1

assertion failure
_GA[6] Assert(1 E ⍺⍺⍨)¨V¨BV⌈/D¨⍵             ⍝   unitary vectors square to 1
       ∧


The `Pq` function cannot be used as a geometric product in a geometric algebra. But that is only because our definition specifically required that vectors squared to `1`.

Our definition of vector inverses, which we came up with intuitively deducing what should be the inverse of a velocity, was a bit shortsighted. It worked well with velocities, but vectors can represent anything as long as the properties defined in `VS` are fulfilled. For example, a vector could represent a rotation, and in that case the inverse should be a rotation in the opposite direction, not a rotation of inverse magnitude. More generally, any component of a multivector or any linear combination of them will fulfill the vector space properties and could be part of the base of a vector space.

We need to fix our geometric algebra definition. But, first, let's see another kind of vectors that do not have a positive metric, neither a negative metric.

### Null vectors

Vectors for which the square is zero will be called *null vectors*. Null vectors can be obtained adding a trivector and a vector orthogonal to the trivector. For example:

In [57]:
t ← 0,zyx   ⍝ vector orthogonal to zyx
n ← zyx,1   ⍝ null vector
'zyx' 't' 'zyx +.×_V t' 'n ← zyx + t' '∆⍨ n' Table zyx t (zyx+.×_V t) n (∆⍨n)
Assert 0 E zyx +.×_V t   ⍝ zyx and t are perpendicular (dot product zero)
Assert 0 E ∆⍨n           ⍝ the square of zyx+t is zero
a b VS u v n             ⍝ n is a vector

┌───────────────┬─────────────────┬───────────┬─────────────────┬───────────────────────────────┐
│zyx │t │zyx +.×_V t│n ← zyx + t │∆⍨ n │
├───────────────┼─────────────────┼───────────┼─────────────────┼───────────────────────────────┤
│0 0 0 0 0 0 0 1│0 0 0 0 0 0 0 0 1│0 │0 0 0 0 0 0 0 1 1│0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0│
└───────────────┴─────────────────┴───────────┴─────────────────┴───────────────────────────────┘

We can also use null vectors as the basis of a vector space and define a product table. However, it is starting to get complicated having to change into a basis of vectors of positive metric. For example, for an algebra with three vectors of negative metric and one null vector, we would need `2` dimensions for each vector of negative metric and `3+1` dimensions for each null vector. To represent the trivector we will need `2*2+3+1` components, so we will have to use 64 components multivectors. However, if we directly used the right basis, we would only need 8 components (`2*3`). Signed geometric algebras allow us to define geometric algebras with a basis that contains vectors of non-positive metric.

## Signature

We will specify what kind of vectors we are using as the basis of a geometric algebra with the *signature* of the algebra. The signature is a list of up to three numbers: the first one specifies the number of vectors with metric `1`, the second one the number of vectors with metric `¯1`, and the third one the number of null vectors, with metric `0`. If there are missing items, it is assumed that there are zero vectors of that kind. A geometric algebra with a specific signature will be called a *signed geometric algebra*.

For example, the algebra of quaternions will correspond to a geometric algebra with signature `0 2` (two base vectors of metric `¯1`). The algebra with three vectors of negative metric and a null vector would have the signature `0 3 1`, and the 3D geometric algebra we saw before would have signature `3` (or `3 0 0`). Other examples of signed geometric algebras will be presented later.

In [58]:
MS ← ⊃1 ¯1 0,.⍴⍨⊢↑⍨3⌈≢   ⍝ (M)etric from (S)ignature ⍵
⍝ eg
ss ← 3 (0 2) (3 0 1) (1 1 1)   ⍝ signatures
's' 'MS s' Table (⍪ss) (⍪MS¨ss)
Assert (1 ¯1 0) ≡ MS 1 1 1

┌───────┬─────────┐
│s │MS s │
├───────┼─────────┤
│┌─────┐│┌───────┐│
││3 │││1 1 1 ││
│├─────┤│├───────┤│
││0 2 │││¯1 ¯1 ││
│├─────┤│├───────┤│
││3 0 1│││1 1 1 0││
│├─────┤│├───────┤│
││1 1 1│││1 ¯1 0 ││
│└─────┘│└───────┘│
└───────┴─────────┘

We also redefine the `_GA` operator such that it only checks that the result is a scalar (ie. that the multivector only have a first component).

In [59]:
]dinput
_GA ← {                                               ⍝ Geometric Algebra with geometric product ⍺⍺
    _ ← Assert (a b ← ⍺)        VS (u v w ← ⍵)        ⍝   vector space: a b scalars, u v w vectors
    _ ← Assert (u ⍺⍺ v ⍺⍺ w)    E w ⍺⍺⍨ u ⍺⍺ v        ⍝   associative product
    _ ← Assert (u∘⍺⍺ ⊃+_V/ v w) E (⊃+_V/ u∘⍺⍺¨ v w)   ⍝   distributive by left
    _ ← Assert (⍺⍺∘u ⊃+_V/ v w) E (⊃+_V/ ⍺⍺∘u¨ v w)   ⍝   distributive by right
    _ ← Assert (a ⍺⍺ v ⍺⍺ b)    E (a × b) × v         ⍝   extension of scalar product
        Assert (⊢ E ⍬⍴⊢)∘(⍺⍺⍨)¨ V¨BV⌈/D¨⍵             ⍝   product of parallel vectors is scalar
}

In [60]:
a b Pq _GA u v w

Quaternions are a signed geometric algebra.

Now, we are ready to give a proper definition of the geometric product, based on the properties checked inside the `_GA` operator:

<div style="border:solid 2px orange; margin:1em; padding:1ex; text-align:center">
    <emph><b>Geometric product</b></emph><br>
    Associative and distributive product that extends the product by scalars to vectors such that the product of parallel vectors is a scalar
</div>

## Signed geometric product

Instead of finding the multiplication table using multivectors with a potentially large number of additional dimensions, we will directly calculate it using the signature of the algebra. The `BS` function returns the bub given the signature. We also define a new operator `_FM` to find a multiplication factor for each bub product given the metric. We need to check if two bub's being multiplied contain the same vector (`⍺∧⍵`), and multiply by the corresponding metric component. The `TS` function is analogous to `TD`, but it takes as argument a signature.

In [61]:
 BS ← BB 2*+/                       ⍝ (B)inary unitary base from (S)ignature
_FM ← {⍺(SB×(⌽⍺⍺)×.*∧)⍵}            ⍝ (F)actor that multiplies product. ⍺⍺: (M)etric, ⍺ ⍵ bub
 TS ← {(∘.PB,⍥⊂∘.((MS⍵)_FM))⍨BS⍵}   ⍝ multiplication (T)able from (S)ignature
⍝ eg
'2' '0 2' '1 0 1' Table TS¨2 (0 2) (1 0 1)   ⍝ multiplication tables for some 2D algebras
'3' '1 1 1'       Table TS¨3 (1 1 1)         ⍝ multiplication tables for some 3D algebras

┌───────────────────┬────────────────────┬───────────────────┐
│2 │0 2 │1 0 1 │
├───────────────────┼────────────────────┼───────────────────┤
│┌───────┬─────────┐│┌───────┬──────────┐│┌───────┬─────────┐│
││0 1 2 3│1 1 1 1│││0 1 2 3│1 1 1 1│││0 1 2 3│1 1 1 1││
││1 0 3 2│1 1 ¯1 ¯1│││1 0 3 2│1 ¯1 ¯1 1│││1 0 3 2│1 1 ¯1 ¯1││
││2 3 0 1│1 1 1 1│││2 3 0 1│1 1 ¯1 ¯1│││2 3 0 1│1 1 0 0││
││3 2 1 0│1 1 ¯1 ¯1│││3 2 1 0│1 ¯1 1 ¯1│││3 2 1 0│1 1 0 0││
│└───────┴─────────┘│└───────┴──────────┘│└───────┴─────────┘│
└───────────────────┴────────────────────┴───────────────────┘

┌───────────────────────────────────────┬─────────────────────────────────────┐
│3 │1 1 1 │
├───────────────────────────────────────┼─────────────────────────────────────┤
│┌───────────────┬─────────────────────┐│┌───────────────┬───────────────────┐│
││0 1 2 3 4 5 6 7│1 1 1 1 1 1 1 1│││0 1 2 3 4 5 6 7│1 1 1 1 1 1 1 1││
││1 0 3 2 5 4 7 6│1 1 ¯1 ¯1 ¯1 ¯1 1 1│││1 0 3 2 5 4 7 6│1 1 ¯1 ¯1 ¯1 ¯1 1 1││
││2 3 0 1 6 7 4 5│1 1 1 1 ¯1 ¯1 ¯1 ¯1│││2 3 0 1 6 7 4 5│1 1 ¯1 ¯1 ¯1 ¯1 1 1││
││3 2 1 0 7 6 5 4│1 1 ¯1 ¯1 1 1 ¯1 ¯1│││3 2 1 0 7 6 5 4│1 1 1 1 1 1 1 1││
││4 5 6 7 0 1 2 3│1 1 1 1 1 1 1 1│││4 5 6 7 0 1 2 3│1 1 1 1 0 0 0 0││
││5 4 7 6 1 0 3 2│1 1 ¯1 ¯1 ¯1 ¯1 1 1│││5 4 7 6 1 0 3 2│1 1 ¯1 ¯1 0 0 0 0││
││6 7 4 5 2 3 0 1│1 1 1 1 ¯1 ¯1 ¯1 ¯1│││6 7 4 5 2 3 0 1│1 1 ¯1 ¯1 0 0 0 0││
││7 6 5 4 3 2 1 0│1 1 ¯1 ¯1 1 1 ¯1 ¯1│││7 6 5 4 3 2 1 0│1 1 1 1 0 0 0 0││
│└───────────────┴─────────────────────┘│└───────────────┴───────────────────┘│
└───────────────────────────────────────┴─────────────────────────────────────┘

We can directly use the tables generated by `TS` with the `_∆_` operator previously defined:

In [62]:
_∆Q ← _∆_(TS 0 2)   ⍝ quaternion product operator
 ∆Q ← ×_∆Q          ⍝ quaternion product
i j ← V¨ 1 (0 1) ⋄ k ← i ∆Q j                     ⍝ quaternion units
'i' 'j' 'k' '∘.∆Q' Table i j k (Z∘C¨∘.∆Q⍨i j k)   ⍝ quaternions multiplication table

┌───┬─────┬────────┬─────────────────────────┐
│i │j │k │∘.∆Q │
├───┼─────┼────────┼─────────────────────────┤
│0 1│0 0 1│0 0 0 ¯1│┌───────┬────────┬──────┐│
│ │ │ ││¯1 │0 0 0 ¯1│0 0 ¯1││
│ │ │ │├───────┼────────┼──────┤│
│ │ │ ││0 0 0 1│¯1 │0 1 ││
│ │ │ │├───────┼────────┼──────┤│
│ │ │ ││0 0 1 │0 ¯1 │¯1 ││
│ │ │ │└───────┴────────┴──────┘│
└───┴─────┴────────┴─────────────────────────┘

Now, it is easy to define different geometric products:

In [63]:
⍝ geometric algebras
Assert a b ∆Q _GA i j k
Assert a b ×_∆_(TS 1 1 1)_GA V¨ u v w
Assert a b ×_∆_(TS 3 0 1)_GA V¨ u (v,1 RC 5) w

## Additional functions

### Exterior and inner products

There are other products that we can define, derived from the geometric product. The *exterior product* (which comes from [Grassman algebra](https://en.wikipedia.org/wiki/Exterior_algebra)) allows the formation of higher-dimensional entities when applied to orthonormal vectors, like the geometric product, but is always zero for parallel vectors. On the contrary, the *inner product* is equal to the geometric product for parallel vectors (and, therefore, equal to the dot product for vectors of positive metric), but it is zero for orthonormal vectors.

We are going to define operators to perform exterior and inner products. Both the exterior and inner product dyadic operators `_∆X_` and `_∆I_` take a product function as left operand and a signature as right operand. The operator `_∆X_` just relays on `_∆_` to do all the work, and uses the `TX` function to calculate a product table in which all vectors are null vectors. It also calls `Z` to simplify the result. The operator `_∆I_` is a bit more complicated. It might use a similar technique to `_∆X_` but, since most elements in the product table will be zero, a more direct method is used. The function `TI` does not actually calculate analogous tables to `TX` or `TS`, but only the diagonal elements of the factors matrix.

In [64]:
⍝ exterior product
 TX  ← TS 0 0,+/                       ⍝ (T)able for e(X)terior product
_∆X_ ← {Z⍺(⍺⍺_∆_(TX⍵⍵))⍵}              ⍝ e(X)terior product, ⍺⍺ is product function, ⍵⍵ is signature
_∆X  ← _∆X_ D                          ⍝ e(X)terior product, ⍺⍺ is product function (assume metric 1)
 ∆X  ← ×_∆X                            ⍝ e(X)terior product (assume metric 1)
⍝ inner product
 TI  ← 0 0⍉∘⊃⌽∘TS                      ⍝ (T)able for (I)nner product
_∆I_ ← {0 V _X ⍺((TI⍵⍵)×_X⍺⍺_Z¨_X)⍵}   ⍝ (I)nner product, ⍺⍺ is product function, ⍵⍵ is signature
_∆I  ← _∆I_ D                          ⍝ (I)nner product, ⍺⍺ is product function (assume metric 1)
 ∆I  ← ×_∆I                            ⍝ (I)nner product (assume metric 1)
⍝ eg
x y z
'∘.∆X' '∘.∆I' Table (∘.∆X⍨ V¨ x y z) (,¨∘.∆I⍨ V¨ x y z)
'∘.(×_∆X_ 1 1 1)' '∘.(×_∆I_ 1 1 1)' Table (∘.(×_∆X_ 1 1 1)⍨ V¨ x y z) (,¨∘.(×_∆I_ 1 1 1)⍨ V¨ x y z)

┌─┬───┬─────┐
│1│0 1│0 0 1│
└─┴───┴─────┘

┌──────────────────────────────────────────┬───────┐
│∘.∆X │∘.∆I │
├──────────────────────────────────────────┼───────┤
│┌───────────┬─────────────┬──────────────┐│┌─┬─┬─┐│
││0 │0 0 0 ¯1 │0 0 0 0 0 ¯1 │││1│0│0││
│├───────────┼─────────────┼──────────────┤│├─┼─┼─┤│
││0 0 0 1 │0 │0 0 0 0 0 0 ¯1│││0│1│0││
│├───────────┼─────────────┼──────────────┤│├─┼─┼─┤│
││0 0 0 0 0 1│0 0 0 0 0 0 1│0 │││0│0│1││
│└───────────┴─────────────┴──────────────┘│└─┴─┴─┘│
└──────────────────────────────────────────┴───────┘

┌──────────────────────────────────────────┬───────────────┐
│∘.(×_∆X_ 1 1 1) │∘.(×_∆I_ 1 1 1)│
├──────────────────────────────────────────┼───────────────┤
│┌───────────┬─────────────┬──────────────┐│┌─┬──┬─┐ │
││0 │0 0 0 ¯1 │0 0 0 0 0 ¯1 │││1│0 │0│ │
│├───────────┼─────────────┼──────────────┤│├─┼──┼─┤ │
││0 0 0 1 │0 │0 0 0 0 0 0 ¯1│││0│¯1│0│ │
│├───────────┼─────────────┼──────────────┤│├─┼──┼─┤ │
││0 0 0 0 0 1│0 0 0 0 0 0 1│0 │││0│0 │0│ │
│└───────────┴─────────────┴──────────────┘│└─┴──┴─┘ │
└──────────────────────────────────────────┴───────────────┘

Alternatively, we could have defined the inner and exterior products in basis of the geometric product:

In [65]:
Assert ∘.((∆I E 2÷⍨∆+_V ∆⍨)⍥V)⍨ u v w
Assert ∘.((∆X E 2÷⍨∆-_V ∆⍨)⍥V)⍨ u v w

The exterior and inner products, when added together, define the geometric product:

In [66]:
Assert ∘.((∆ E ∆X +_V ∆I)⍥V)⍨ u v w

### Reverses and complements

The [reverse](http://rigidgeometricalgebra.org/wiki/index.php?title=Reverses) is a linear operation, so it can be applied component by component. The reverse of an n-vector that constitutes the product of n 1-vectors is the result of multiplying those same n 1-vectors in the reverse order (from right to left instead of from left to right). The result is that each component must be multiplied by `1` or by `¯1`. This can be calculated simply as the result of squaring each component of the base (`SB⍨BB`):

In [67]:
R ← ×∘(SB⍨¨BB∘≢)⍨   ⍝ (R)everse
⍝ eg
i←0.1,⍎¨1↓(↓↑BB 16)(⊢⍤/¨)⊂'4321'
'i' 'R i' Table ⍪¨i (R i)
Assert E∘R∘R⍨¨ i u v w   ⍝ reverse of reverse is the same

┌──────┬──────┐
│i │R i │
├──────┼──────┤
│ 0.1│ 0.1│
│ 1 │ 1 │
│ 2 │ 2 │
│ 21 │ ¯21 │
│ 3 │ 3 │
│ 31 │ ¯31 │
│ 32 │ ¯32 │
│ 321 │¯321 │
│ 4 │ 4 │
│ 41 │ ¯41 │
│ 42 │ ¯42 │
│ 421 │¯421 │
│ 43 │ ¯43 │
│ 431 │¯431 │
│ 432 │¯432 │
│4321 │4321 │
└──────┴──────┘

Notice that, as we have done with products, we can improve the performance of the reverse operation using a precomputed table:

In [68]:
TR ← SB⍨¨BB    ⍝ (T)able to (R)everse multivectors of ⍵ components
 R ← ×∘TR∘≢⍨   ⍝ (R)everse (calculate table)
⍝ eg
tr←TR≢us
]RunTime 'tr×us' ' R us' -compare

tr×us → 5.3E¯7 | 0% ⎕ 
 R us → 1.5E¯5 | +2682% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

Like reverses, [complements](http://rigidgeometricalgebra.org/wiki/index.php?title=Complements) are linear operations.

The left complement of a multivector is the multivector that multiplied by the left will result in the *pseudoscalar*. The pseudoscalar for n dimensions is the last component of the multivector, corresponding to the product of n 1-vectors.

The right complement is the multivector that multiplied by the right will result in the pseudoscalar. And the double complement can be calculated either as the result of applying twice the left complement or as the result of applying twice the right complement (the result will be the same).

Before defining the functions to find the complement, we define an operator `_C` which calculates the products using the given function as product. It calculates the corresponding pseudoscalars. The left and right complements are then found using as operand either the geometric product, or its reverse:

In [69]:
_C ← {⍺←D⍵ ⋄ (⌽×_V+/⍤⍺⍺¨∘⌽⍨∘BV∘≢)⍵ X⍨2*⍺}   ⍝ (C)omplement
LC ← ∆ _C                                   ⍝ (L)eft complement
RC ← ∆⍨_C                                   ⍝ (R)ight complement
DC ← LC⍣2                                   ⍝ (D)ouble complement
⍝ eg
'i' 'LC i' 'RC i' 'DC i' Table ⍪¨ i (LC i) (RC i) (DC i)
p ← (1↑⍨∘-2*∘⌈2⍟≢)¨bu          ⍝ pseudoscalars
Assert p E∘(∆⍨∘LC⍨)¨bu         ⍝ Left-product by LC is pseudoscalar
Assert p E∘(∆∘RC⍨)¨bu          ⍝ right-product by LC is pseudoscalar
Assert (zyx E ∆⍨∘(3∘LC)⍨)¨bu   ⍝ left-product by 8∘LC is the 3D pseudoscalar
Assert (zyx E ∆∘(3∘LC)⍨)¨bu    ⍝ right-product by 8∘LC is the 3D pseudoscalar
Assert E∘RC∘LC⍨¨ u v w         ⍝ right complement of left complement is the same
Assert (DC E RC⍣2)¨ u v w      ⍝ double left complement and double right complement are the same

┌──────┬──────┬──────┬──────┐
│i │LC i │RC i │DC i │
├──────┼──────┼──────┼──────┤
│ 0.1│4321 │4321 │ 0.1│
│ 1 │¯432 │ 432 │ ¯1 │
│ 2 │ 431 │¯431 │ ¯2 │
│ 21 │ 43 │ 43 │ 21 │
│ 3 │¯421 │ 421 │ ¯3 │
│ 31 │ ¯42 │ ¯42 │ 31 │
│ 32 │ 41 │ 41 │ 32 │
│ 321 │ ¯4 │ 4 │¯321 │
│ 4 │ 321 │¯321 │ ¯4 │
│ 41 │ 32 │ 32 │ 41 │
│ 42 │ ¯31 │ ¯31 │ 42 │
│ 421 │ 3 │ ¯3 │¯421 │
│ 43 │ 21 │ 21 │ 43 │
│ 431 │ ¯2 │ 2 │¯431 │
│ 432 │ 1 │ ¯1 │¯432 │
│4321 │ 0.1│ 0.1│4321 │
└──────┴──────┴──────┴──────┘

### Antiproducts

The geometric antiproduct is the dual of the geometric product. Analogously, there are inner and exterior antiproducts. Products and antiproducts are related through the [De Morgan laws](https://rigidgeometricalgebra.org/wiki/index.php?title=Geometric_products#De_Morgan_Laws), which we use to define an antiproduct operator that can be applied to the different products.

In [70]:
_⍙_ ← {⍵⍵ LC ⍺ ⍺⍺⍥((⍺(⍵⍵⍨⍨)⍵)∘RC) ⍵}   ⍝ ⍺⍺ is product, ⍵⍵ is dimensions
⍝ eg
⍙ ← ∆ _⍙_ 3                            ⍝ 3D anti-geometric-product
Assert (3 LC u ∆ v) E u ⍙⍥(3∘LC) v
Assert (3 LC u ⍙ v) E u ∆⍥(3∘LC) v
Assert (3 RC u ∆ v) E u ⍙⍥(3∘RC) v
Assert (3 RC u ⍙ v) E u ∆⍥(3∘RC) v
∆I ← 3 _∆I ⋄ ⍙I ← ∆I _⍙_ 3             ⍝ 3D anti-inner-product
Assert (3 LC u ∆I v) E u ⍙I⍥(3∘LC) v
Assert (3 LC u ⍙I v) E u ∆I⍥(3∘LC) v
Assert (3 RC u ∆I v) E u ⍙I⍥(3∘RC) v
Assert (3 RC u ⍙I v) E u ∆I⍥(3∘RC) v
∆X ← ×_∆X_ 3 ⋄ ⍙X ← ∆X _⍙_ 3           ⍝ 3D anti-exterior-product
Assert (3 LC u ∆X v) E u ⍙X⍥(3∘LC) v
Assert (3 LC u ⍙X v) E u ∆X⍥(3∘LC) v
Assert (3 RC u ∆X v) E u ⍙X⍥(3∘RC) v
Assert (3 RC u ⍙X v) E u ∆X⍥(3∘RC) v

### Inverse

The problem is to find the multiversor that, when multiplied by the right, returns the scalar unit. At difference of reverses and complements, the inverse is not a linear operation.

To find the solution, we solve a linear system using the function `⌹`. To apply it, we want a full matrix, so we will use an unscalar function (`US`) first. This function will take a multivector with scalar components and extend them such that all components have the same shape (for example, `US(1 2)0` results in `(1 2)(0 0)`).

In [71]:
US ← ×∘(×/⍤=⍨)⍨  ⍝ (U)n(S)calar
_I ← {⍺←1 ⋄ Z∘C ⍺ ⍺⍺ ⊂∘⍉⍤¯1⍉(↑∘1∘≢⌹⍤2(↑[¯0.5]⍣2⊂⍺⍺¨BV∘≢))US⍵X⍨2*D⍵}
∆∆ ← ∆ _I
⍝ eg
' ' '∆∆' '∆∆⍨' Table ⍪¨ i (∆∆ i) (∆∆⍨i)
Assert 1 E¨ ∆∆⍨¨u v w

┌──────┬─────────┬───┐
│ │∆∆ │∆∆⍨│
├──────┼─────────┼───┤
│ 0.1│ 1.821E¯8│1 │
│ 1 │¯1.830E¯7│ │
│ 2 │¯1.238E¯7│ │
│ 21 │¯1.179E¯6│ │
│ 3 │¯1.770E¯7│ │
│ 31 │¯1.708E¯6│ │
│ 32 │¯1.773E¯6│ │
│ 321 │ 1.781E¯5│ │
│ 4 │¯2.342E¯7│ │
│ 41 │¯2.285E¯6│ │
│ 42 │¯2.330E¯6│ │
│ 421 │ 2.336E¯5│ │
│ 43 │¯2.387E¯6│ │
│ 431 │ 2.391E¯5│ │
│ 432 │ 2.397E¯5│ │
│4321 │ 2.397E¯4│ │
└──────┴─────────┴───┘

When we try to inverse a 0=vector, we will get a domain error.

In [72]:
∆∆ n   ⍝ ERROR: 0=vectors produce domain error

DOMAIN ERROR
_I[0] _I←{⍺←1 ⋄ Z∘C ⍺ ⍺⍺⊂∘⍉⍤¯1⍉(↑∘1∘≢⌹⍤2(↑[¯0.5]⍣2⊂⍺⍺¨BV∘≢))US ⍵ X⍨2*D ⍵}
                               ∧


## Geometric algebra namespace

It is clear that using a precomputed multiplication table has several advantages. In order to make it easier to define a geometric product and, more generally, to solve geometric problems, we are going to use geometric algebra namespaces. A GA namespace will store the properties of the algebra and all related functions for a given signature.

The `GA` function takes a signature and returns a namespace that contains all the needed tables, as well as the corresponding functions and operators. The namespace must also contain a number of helper functions from those that we have previously defined.

In [73]:
]dinput
GA ← {                                                                  ⍝ GEOMETRIC ALGEBRA
        ns ← '_Z' '_X' 'Z' 'S' 'US' 'C' 'V' '_V' 'D'
        ns ,←'BS' 'BV' 'TS' 'TX' 'TI' 'TR' '_FM' 'MS'  
        g ← ⎕NS ns ⋄ g.s g.b ← ⍵ (S BV 2*d←+/⍵)  ⋄  g._⍙←_⍙_ d          ⍝   namespace, signature, base, anti
        g._∆ ← _∆_(TS⍵)    ⋄  g._∆X ← _∆_(TX⍵)   ⋄  g._∆I ← _∆I_(TI⍵)   ⍝   geometric, exterior, inner operators
        g.∆  ← ×g._∆       ⋄  g.∆X  ← ×g._∆X     ⋄  g.∆I  ← ×g._∆I      ⍝   products
        g.∆V ← g.∆⍥V       ⋄  g.∆XV ← g.∆X⍥V     ⋄  g.∆IV ← g.∆I⍥V      ⍝   products of vectors
        g.∆v ← g.∆∘V       ⋄  g.∆Xv ← g.∆X∘V     ⋄  g.∆Iv ← g.∆I∘V      ⍝   products by vector
        g.⍙  ← g.∆ g._⍙    ⋄  g.⍙X  ← g.∆X g._⍙  ⋄  g.⍙I  ← g.∆I g._⍙   ⍝   antiproducts
        g.∆∆ ← g.∆ _I      ⋄  g.∆∆V ← g.∆∆⍥V     ⋄  g.∆∆v ← g.∆∆∘V      ⍝   inverse and division
        g.LC ← g.∆ _C      ⋄  g.RC  ← g.∆⍨_C     ⋄  g.DC  ← g.LC⍣2      ⍝   complements: left, right, double
        g.R  ← ×∘(TR 2*d)  ⋄  g.X   ← X{⍺←≢b ⋄ ⍺(⍺⍺)⍵}  ⋄  g            ⍝   reverse and extend
}

In [74]:
⍝ eg
g3 ← GA 3
Assert a b (g3.∆ _GA) V¨ u v w
'signature' 'table' 'base' 'eg: u g3.∆ v' Table g3.s (TS g3.s) (⍪g3.b) (u g3.∆ v)

┌─────────┬───────────────────────────────────────┬─────────────────┬────────────────────────────────┐
│signature│table │base │eg: u g3.∆ v │
├─────────┼───────────────────────────────────────┼─────────────────┼────────────────────────────────┤
│3 │┌───────────────┬─────────────────────┐│┌───────────────┐│┌──────┬───────┬──────┬────────┐│
│ ││0 1 2 3 4 5 6 7│1 1 1 1 1 1 1 1│││1 │││ 5.314│ 0.4577│ 6.884│ 0.6044 ││
│ ││1 0 3 2 5 4 7 6│1 1 ¯1 ¯1 ¯1 ¯1 1 1││├───────────────┤││18.21 │21.42 │13.42 │10.66 ││
│ ││2 3 0 1 6 7 4 5│1 1 1 1 ¯1 ¯1 ¯1 ¯1│││0 1 │││ 6.296│ 3.927 │10.53 │10.18 ││
│ ││3 2 1 0 7 6 5 4│1 1 ¯1 ¯1 1 1 ¯1 ¯1││├───────────────┤││ 1.252│ 0.7338│ 1.016│ 0.02546││
│ ││4 5 6 7 0 1 2 3│1 1 1 1 1 1 1 1│││0 0 1 │││15.99 │ 0.1252│ 5.059│13.17 ││
│ ││5 4 7 6 1 0 3 2│1 1 ¯1 ¯1 ¯1 ¯1 1 1││├───────────────┤│└──────┴───────┴──────┴────────┘│
│ ││6 7 4 5 2 3 0 1│1 1 1 1 ¯1 ¯1 ¯1 ¯1│││0 0 0 1 ││ │
│ ││7 6 5 4 3 2 1 0│1 1 ¯1 ¯1 1 1 ¯1 ¯1││├───────────────┤│ │
│ │└───────────────┴─────────────────────┘││0 0 0 0 1 ││ │
│ │ │├───────────────┤│ │
│ │ ││0 0 0 0 0 1 ││ │
│ │ │├───────────────┤│ │
│ │ ││0 0 0 0 0 0 1 ││ │
│ │ │├───────────────┤│ │
│ │ ││0 0 0 0 0 0 0 1││ │
│ │ │└───────────────┘│ │
└─────────┴───────────────────────────────────────┴─────────────────┴────────────────────────────────┘

# Geometric algebras

Using our new `GA` function, it is easy to define different geometric algebras and access the corresponding operations.

## 0D / 1D / 2D / 3D vectors

We can define algebras for vector spaces of an arbitrary number of dimensions.

The 0-dimensional vector space will be the scalar field (usually the real numbers; in APL, just numbers). The inverse and multiplication function must work as `÷` and `×`, and using enclosed arrays, scalar extension should be applied:

In [75]:
g0 ← GA 0
'a' 'b' 'a g0.∆ b' 'a g0.∆∆ b' Table a b (a g0.∆ b) (a g0.∆∆ b)
Assert a b (g0.∆ _GA) ⊂¨v
Assert a (⊂⍤× E g0.∆) b
Assert a (⊂⍤÷ E g0.∆∆) b

┌────────┬────────┬────────┬─────────┐
│a │b │a g0.∆ b│a g0.∆∆ b│
├────────┼────────┼────────┼─────────┤
│┌──────┐│┌──────┐│┌──────┐│┌───────┐│
││3.309 │││0.4005│││ 1.325│││8.263 ││
││4.118 │││1.996 │││ 8.221│││2.063 ││
││2.876 │││4.992 │││14.36 │││0.5762 ││
││4.699 │││4.221 │││19.83 │││1.113 ││
││0.3687│││4.918 │││ 1.813│││0.07498││
│└──────┘│└──────┘│└──────┘│└───────┘│
└────────┴────────┴────────┴─────────┘

DOMAIN ERROR
V[0] V←{⍺←2*⍳≢⍵ ⋄ r←⍺⌊⍥⍴⍵ ⋄ v←0⍴⍨1+⌈/,a←r↑⍺ ⋄ v[a]+←r↑⍵ ⋄ v}  ⍝ multi(V)ector with components ⍵ at ⍺ (default vector)
                               ∧


When we add a vector, a proper vector with metric 1, to have an 1-dimensional space, we will get the [hyperbolic numbers](https://en.wikipedia.org/wiki/Split-complex_number):

In [76]:
g1 ← GA 1
'a' 'b' 'g1.∆⍨ a b' Table a b (g1.∆⍨ a b)
Assert a b (g1.∆ _GA) 2↑¨ u v w

┌────────┬────────┬──────────────────┐
│a │b │g1.∆⍨ a b │
├────────┼────────┼──────────────────┤
│┌──────┐│┌──────┐│┌───────┬────────┐│
││3.309 │││0.4005│││┌─────┐│┌──────┐││
││4.118 │││1.996 ││││11.11│││ 2.651│││
││2.876 │││4.992 ││││20.94│││16.44 │││
││4.699 │││4.221 ││││33.19│││28.72 │││
││0.3687│││4.918 ││││39.89│││39.67 │││
│└──────┘│└──────┘│││24.32│││ 3.626│││
│ │ ││└─────┘│└──────┘││
│ │ │└───────┴────────┘│
└────────┴────────┴──────────────────┘

With two dimensions, things start getting interesting. Now we have vectors on a plane, and we can rotate them!

In [77]:
g2 ← GA 2
Assert a b (g2.∆ _GA) u v w
r90 ← 0 1   g2.∆∆⍥V 1 ⋄ Assert (V 0 1)   E r90 g2.∆   V 1   ⍝   0 1 is 90 deg rotation of 1 0
r45 ← 1 1 U⍤g2.∆∆⍥V 1 ⋄ Assert (V U 1 1) E r45 g2.∆   V 1   ⍝ U 1 1 is 45 deg rotation of 1 0 
                        Assert (V 0 1)   E r45 g2.∆⍣2 V 1   ⍝   0 1 is 2 45 deg rotations of 1 0

We can also use bivectors as areas:

In [78]:
'a' 'b' 's←a g2.∆⍥V 0,b' 's g2.∆∆∘V 0,b' Table a b (a g2.∆V 0,b) ((0,b) g2.∆∆∘V⍨ a g2.∆V 0,b)

┌────────┬────────┬───────────────┬─────────────┐
│a │b │s←a g2.∆⍥V 0,b │s g2.∆∆∘V 0,b│
├────────┼────────┼───────────────┼─────────────┤
│┌──────┐│┌──────┐│┌─┬─┬─┬───────┐│┌─┬──────┐ │
││3.309 │││0.4005│││0│0│0│ ¯1.325│││0│3.309 │ │
││4.118 │││1.996 │││0│0│0│ ¯8.221│││ │4.118 │ │
││2.876 │││4.992 │││0│0│0│¯14.36 │││ │2.876 │ │
││4.699 │││4.221 │││0│0│0│¯19.83 │││ │4.699 │ │
││0.3687│││4.918 │││0│0│0│ ¯1.813│││ │0.3687│ │
│└──────┘│└──────┘│└─┴─┴─┴───────┘│└─┴──────┘ │
└────────┴────────┴───────────────┴─────────────┘

The 3D space is pretty much where we live, and has already been discussed in a few examples, so you should be familiar with this one by now. We have scalars, vectors (lengths), bivectors (areas), and the pseudoscalar is a trivector (volume):

In [79]:
id Table g3.b
u1←V 2 3 1 ⋄ u2←V 5 3 2 ⋄ 'uv∆' Table u1 u2 (u1 g3.∆ u2)

┌─┬───┬─────┬───────┬─────────┬───────────┬─────────────┬───────────────┐
│1│x │y │yx │z │zx │zy │zyx │
├─┼───┼─────┼───────┼─────────┼───────────┼─────────────┼───────────────┤
│1│0 1│0 0 1│0 0 0 1│0 0 0 0 1│0 0 0 0 0 1│0 0 0 0 0 0 1│0 0 0 0 0 0 0 1│
└─┴───┴─────┴───────┴─────────┴───────────┴─────────────┴───────────────┘

┌─────────┬─────────┬─────────────────┐
│u │v │∆ │
├─────────┼─────────┼─────────────────┤
│0 2 3 0 1│0 5 3 0 2│21 0 0 9 0 1 ¯3 0│
└─────────┴─────────┴─────────────────┘

## Complex numbers

Complex numbers are obtained by the addition of a real number (a scalar in APL) and an imaginary number. Although Dyalog APL supports complex numbers, we can also represent them defining a geometric algebra with the signature 0 1:

In [80]:
c ← GA 0 1
'C' 'GA 0 1' Table (,¨∘.×⍨ 1 0j1) (∘.c.∆⍨ c.b)
Assert (∘.×⍨ 1 0j1) ≡ (⊂1 0j1)+.×∘c.X¨∘.c.∆⍨ c.b
'c' '∆∆ c' '∆∆⍨c' Table (c.X v) (∆∆ c.X v) (∆∆⍨c.X v)
Assert (9 11○1÷1j1) ≡ c.∆∆ 1 1
Assert (9 11○1 ÷ 1 0j1 +.× c.X v) ≡ c.∆∆ c.X v

┌─────────┬──────────┐
│C │GA 0 1 │
├─────────┼──────────┤
│┌───┬───┐│┌───┬────┐│
││1 │0J1│││1 │0 1 ││
│├───┼───┤│├───┼────┤│
││0J1│¯1 │││0 1│¯1 0││
│└───┴───┘│└───┴────┘│
└─────────┴──────────┘

┌───────────────┬──────────────────┬────┐
│c │∆∆ c │∆∆⍨c│
├───────────────┼──────────────────┼────┤
│┌──────┬──────┐│┌───────┬────────┐│1 │
││4.991 │0.4499│││ 0.202 │¯0.01821││ │
││3.075 │4.298 │││¯0.341 │ 0.4766 ││ │
││2.979 │3.19 │││¯2.294 │ 2.456 ││ │
││0.4763│0.2869│││ 3.295 │¯1.984 ││ │
││0.9872│2.65 │││¯0.1632│ 0.4381 ││ │
│└──────┴──────┘│└───────┴────────┘│ │
└───────────────┴──────────────────┴────┘

## Quaternions

Dyalog APL does not support quaternions natively, but an implementation is included in the dfns workspace.

In [81]:
_∆Q ← _∆_(TS 0 2)   ⍝ quaternion product operator
 ∆Q ← ×_∆Q          ⍝ quaternion product
i j ← V¨ 1 (0 1) ⋄ k ← i ∆Q j  ⍝ quaternion units
tabq ← 'i' 'j' 'k' '∘.∆Q' Table i j k (Z∘C¨∘.∆Q⍨i j k)
tabh ← ('ijk',⊂'∘.(×H)') Table (⊂∘.(×H)⍨ i j k),⍨i j k ← ↓ ⍎H'0i1 0j1 0k1'
'GA' 'dfns.H' Table tabq tabh
Assert (∘.(×H)⍨ i j k) ≡ (⊂1 1 1 ¯1)×_X¨∘.∆Q⍨ i j (-k)

┌──────────────────────────────────────────────┬──────────────────────────────────────────────────────┐
│GA │dfns.H │
├──────────────────────────────────────────────┼──────────────────────────────────────────────────────┤
│┌───┬─────┬────────┬─────────────────────────┐│┌───────┬───────┬───────┬────────────────────────────┐│
││i │j │k │∘.∆Q │││i │j │k │∘.(×H) ││
│├───┼─────┼────────┼─────────────────────────┤│├───────┼───────┼───────┼────────────────────────────┤│
││0 1│0 0 1│0 0 0 ¯1│┌───────┬────────┬──────┐│││0 1 0 0│0 0 1 0│0 0 0 1│┌────────┬────────┬────────┐││
││ │ │ ││¯1 │0 0 0 ¯1│0 0 ¯1││││ │ │ ││¯1 0 0 0│0 0 0 1 │0 0 ¯1 0│││
││ │ │ │├───────┼────────┼──────┤│││ │ │ │├────────┼────────┼────────┤││
││ │ │ ││0 0 0 1│¯1 │0 1 ││││ │ │ ││0 0 0 ¯1│¯1 0 0 0│0 1 0 0 │││
││ │ │ │├───────┼────────┼──────┤│││ │ │ │├────────┼────────┼────────┤││
││ │ │ ││0 0 1 │0 ¯1 │¯1 ││││ │ │ ││0 0 1 0 │0 ¯1 0 0│¯1 0 0 0│││
││ │ │ │└───────┴────────┴──────┘│││ │ │ │└────────┴────────┴────────┘││
│└───┴─────┴────────┴─────────────────────────┘│└───────┴───────┴───────┴────────────────────────────┘│
└──────────────────────────────────────────────┴──────────────────────────────────────────────────────┘

Both the `×H` and `∆Q` products can be used to obtain the same results. Let's check the performance.

In [82]:
HQ ← {⊂⍤1⍉1 1 1 ¯1×[1]⍵}
qg ← HQ qh ← ?1e4 4⍴0
Assert (∆Q⍨qg) ≡ HQ(×H⍨)qh
]RunTime     '∆Q⍨qg' '×H⍨qh' -compare
]SpaceNeeded '∆Q⍨qg' '×H⍨qh'

assertion failure
      Assert(∆Q⍨qg)≡HQ(×H⍨)qh
      ∧


∆Q⍨qg → 3.1E¯4 | 0% 
* ×H⍨qh → 4.3E¯2 | +13680% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌─────┬───────┐
│∆Q⍨qg│1761020│
├─────┼───────┤
│×H⍨qh│2245354│
└─────┴───────┘

The GA version, which stores multiple values as a vector of columns (instead of a column of vector) is much faster. The difference is even larger when using the outer product:

In [83]:
qg ← HQ qh ← ?10 4⍴0
]RunTime     '∘.×_∆Q⍨qg' '∘.(×H)⍨qh' -compare
]SpaceNeeded '∘.×_∆Q⍨qg' '∘.(×H)⍨qh'

∘.×_∆Q⍨qg → 1.6E¯5 | 0% 
* ∘.(×H)⍨qh → 8.4E¯2 | +539900% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

┌─────────┬──────┐
│∘.×_∆Q⍨qg│18916 │
├─────────┼──────┤
│∘.(×H)⍨qh│120556│
└─────────┴──────┘

## Projective geometric algebra

See: https://projectivegeometricalgebra.org/

In [84]:
pga ← GA 3 0 1
pga.B ← (8⌊≢)↑⊢  ⋄  pga.W ← (8⌊≢)↓⊢  ⍝ bulk and weight
pga.Point ← V,∘1                     ⍝ point
pga.Line  ← {⍺←0 ⋄ ⍺ ∆X ⍵}           ⍝ line from two points (or through origin)
pga.Plane ← {⍺←0 ⋄ ⍺ ∆X ⍵}           ⍝ plane containing line and point (or through origin)
pga.Proj  ← {⍵ ⍙X⍨ ⍺ ∆X⍨ LC W ⍵}     ⍝ projection of ⍺ into ⍵

In [85]:
⍝ eg
(p1 p2 p3) ← pga.Point¨(3 2 1)(4 2 3)(2 3 1)
(line12 line23 line31) ← p1 p2 p3 pga.Line¨ p2 p3 p1
(plane123 plane231 plane312) ← p3 p1 p2 pga.Plane¨ line12 line23 line31
'points' 'lines' 'planes' Table ⍪¨¨(p1 p2 p3) (line12 line23 line31) (plane123 plane231 plane312)

┌───────┬──────────┬──────────┐
│points │lines │planes │
├───────┼──────────┼──────────┤
│┌─┬─┬─┐│┌──┬──┬──┐│┌──┬──┬──┐│
││0│0│0│││ 0│ 0│ 0│││ 0│ 0│ 0││
││3│4│2│││ 0│ 0│ 0│││ 0│ 0│ 0││
││2│2│3│││ 0│ 0│ 0│││ 0│ 0│ 0││
││0│0│0│││ 2│¯8│ 5│││ 0│ 0│ 0││
││1│3│1│││ 0│ 0│ 0│││ 0│ 0│ 0││
││0│0│0│││¯5│ 2│ 1│││ 0│ 0│ 0││
││0│0│0│││¯4│ 7│¯1│││ 0│ 0│ 0││
││0│0│0│││ 0│ 0│ 0│││ 9│ 9│ 9││
││1│1│1│││ 0│ 0│ 0│││ 0│ 0│ 0││
│└─┴─┴─┘││ 1│¯2│ 1│││ 0│ 0│ 0││
│ ││ 0│ 1│¯1│││ 0│ 0│ 0││
│ ││ 0│ 0│ 0│││¯1│¯1│¯1││
│ ││ 2│¯2│ 0│││ 0│ 0│ 0││
│ ││ 0│ 0│ 0│││¯2│¯2│¯2││
│ ││ 0│ 0│ 0│││ 2│ 2│ 2││
│ ││ 0│ 0│ 0│││ 0│ 0│ 0││
│ │└──┴──┴──┘│└──┴──┴──┘│
└───────┴──────────┴──────────┘

# Summary

In [86]:
)clear

clear ws

In [87]:
:Namespace ga                              ⍝ Geometric Algebra
    ⎕IO ← 0                                                               ⍝ important
    ⎕CT ← 1e¯10                                                           ⍝ avoid rounding issues
    Assert ← {⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←1}            ⍝ Assert by Roger Hui

    ⍝ vectors
     M ← 0.5*⍨+.×⍨  ⋄   U ← ×∘(÷M)⍨   ⋄   I ← ×∘(÷+.×⍨)⍨                  ⍝ magnitude, unitary, inverse
    AV ← ⊂⍤1↑[0]    ⋄  VA ← ⊂⍤¯1↑[0]  ⋄  BV ← 1↑⍨¨∘-1+⍳                   ⍝ array from vector, vector from array, base

    ⍝ vector extension
     C ← {⍺←⎕CT ⋄ ⎕CT←⍺ ⋄ 1=≢∪⍵:⊃⍵ ⋄ ⍺∧.>|∊⍵:0 ⋄ ⍵}¨                      ⍝ collapse equal values
     S ← {⍬≡⍵:0 ⋄ (1<≢⍵)∨(0<≡⍵)∧1<≢⊃⍵:⍵ ⋄ ⊃⍵}  ⋄  US ← ×∘(×/⍤=⍨)⍨         ⍝ scalar if possible and unscalar
     Z ← {⍺←⎕CT ⋄ ⎕CT←⍺ ⋄ S⍵↓⍨-⊥⍨(1∧.=1+∊)¨⍵}  ⋄   E ← ≡⍥Z⍥C              ⍝ remove trailing zeros and equivalent
     X ← {⍺←2*D⍵ ⋄ ⍺↑⍵,0⍴⍨0⌈⍺-≢⍵}  ⋄  _X ← {⍺←⊢ ⋄ ⍺(⍺⍺⍥((⍺⌈⍥≢⍵)∘X))⍵}     ⍝ extend and apply extending arguments
    _V ← {⍺←⊢ ⋄ Z∘C ⍺(⍺⍺ _X)⍵}     ⋄  _Z ← {0≡⍺:0 ⋄ 0≡⍵:0 ⋄ ⍺(⍺⍺)⍵}       ⍝ apply vector extension or zero function

    ⍝ multivectors
     V ← {⍺←2*⍳≢⍵ ⋄ r←⍺⌊⍥⍴⍵ ⋄ v←0⍴⍨1+⌈/,a←r↑⍺ ⋄ v[a]+←r↑⍵ ⋄ v}            ⍝ multivector
    BB ← ↓∘⍉2⊥⍣¯1⍳  ⋄  G ← {⌈/+/¨(0≢¨C ⍵)/BB≢⍵}  ⋄  D ← ⌈2⍟⌈⍥≢            ⍝ binary unitary base, grade and dimensions
    GB ← {b←Z({⊂⊂⍵}⌸+/¨BB≢⍵)⌷¨⊂,⍵ ⋄ ⍺←⍳≢b ⋄ (⊂⍺)⌷(1+⌈/,⍺)X b}             ⍝ group in blades

    ⍝ product tables
     PB ← 2⊥≠  ⋄  SB ← ¯1*(≠\¯1↓⊢)≠.∧1↓⊣  ⋄  MS ← ⊃1 ¯1 0,.⍴⍨⊢↑⍨3⌈≢       ⍝ position, sign and metric
   _FM ← {⍺(SB×(⌽⍺⍺)×.*∧)⍵}  ⋄  TS ← {(∘.PB,⍥⊂∘.((MS⍵)_FM))⍨BB 2*+/⍵}     ⍝ factor from metric, tables from signature
     TX ← TS 0 0,+/  ⋄  TI ← 0 0⍉∘⊃⌽∘TS  ⋄  TR ← SB⍨¨BB∘≢  ⋄  BT ← ⊃V¨/   ⍝ exterior, inner, reversion and base

    ⍝ geometric operators
    _∆_  ← {p s←⍺(⍵⍵⍨⍨)⍵ ⋄ (r↑p)V(r↑s)×(r←(≢¨⍺⍵)⌊⍴p)↑⍺∘.⍺⍺⍥,⍵}            ⍝ geometric product
    _∆I_ ← {0 V _X ⍺(⍵⍵×_X⍺⍺¨_X)⍵}        ⋄  _∆X_ ← {Z⍺(⍺⍺_∆_⍵⍵)⍵}        ⍝ inner and exterior product
    _∆   ← _∆_(TS⍤D)  ⋄  _∆X ← _∆_(TX⍤D)  ⋄  _∆I ← _∆I_(TI⍤D)             ⍝ geometric, inner and exterior product
    _⍙_  ← {⍵⍵ LC ⍺ ⍺⍺⍥((⍺(⍵⍵⍨⍨)⍵)∘RC)⍵}  ⋄  _⍙ ← _⍙_ D                   ⍝ antigeometric product
    _I   ← {⍺←1 ⋄ Z∘C⍺ ⍺⍺ ⊂∘⍉⍤¯1⍉(↑∘1∘≢⌹⍤2(↑[-÷2]⍣2⊂⍺⍺¨BV∘≢))US⍵X⍨2*D⍵}   ⍝ inverse
    _C   ← {⍺←D⍵ ⋄ (⌽×_V+/⍤⍺⍺¨∘⌽⍨∘BV∘≢)⍵ X⍨2*⍺}                           ⍝ complement

    ⍝ geometric functions
    ∆  ← ×_Z_∆  ⋄  ∆X ← ×_∆X   ⋄  ∆I ← ×_∆I                               ⍝ geometric, exterior and inner product
    ⍙  ← ∆ _⍙   ⋄  ⍙X ← ∆X _⍙  ⋄  ⍙I ← ∆I _⍙                              ⍝ antigeometric, exterior and inner product
    LC ← ∆ _C   ⋄  RC ← ∆⍨ _C  ⋄  DC ← LC⍣2                               ⍝ left, right and double complement
    ∆∆ ← ∆ _I   ⋄  ⍙⍙ ← ⍙ _I   ⋄  R  ← ×∘TR⍨  ⋄  PS ← 1↑⍨∘-2*∘⌈2⍟≢        ⍝ inverse, antiinverse, reverse, pseudoscalar

    GA ← {                                                                ⍝ GEOMETRIC ALGEBRA
        g ← ⎕NS⍬  ⋄  g.s g.b ← ⍵ (S BV 2*d←+/⍵)                           ⍝   namespace, signature, base
        g._Z←_Z ⋄ g._X←_X ⋄ g.Z←Z ⋄ g.S←S ⋄ g.US←US ⋄ g.C←C
        g.V←V ⋄ g._V←_V ⋄ g.D←D ⋄ g.BV←BV ⋄ g._⍙←_⍙_ d
        g._∆ ← _∆_(TS⍵)  ⋄  g._∆X ← _∆_(TX⍵)   ⋄  g._∆I ← _∆I_(TI⍵)       ⍝   geometric, exterior and inner operator
        g.∆  ← ×g._∆     ⋄  g.∆X  ← ×g._∆X     ⋄  g.∆I  ← ×g._∆I          ⍝   products
        g.∆V ← g.∆⍥V     ⋄  g.∆XV ← g.∆X⍥V     ⋄  g.∆IV ← g.∆I⍥V          ⍝   products of vectors
        g.∆v ← g.∆∘V     ⋄  g.∆Xv ← g.∆X∘V     ⋄  g.∆Iv ← g.∆I∘V          ⍝   products by vector
        g.⍙  ← g.∆ g._⍙  ⋄  g.⍙X  ← g.∆X g._⍙  ⋄  g.⍙I  ← g.∆I g._⍙       ⍝   antiproducts
        g.∆∆ ← g.∆ _I    ⋄  g.∆∆V ← g.∆∆⍥V     ⋄  g.∆∆v ← g.∆∆∘V          ⍝   inverse and division
        g.LC ← g.∆ _C    ⋄  g.RC  ← g.∆⍨_C     ⋄  g.DC  ← g.LC⍣2          ⍝   complements: left, right, double
        g.R  ← ×∘(TR⍵)⍨  ⋄  g.X   ← X{⍺←≢b ⋄ ⍺(⍺⍺)⍵}  ⋄  g                ⍝   reverse and extend
    }

    g0 g1 g2 g3 c q pga ← GA¨(⍳4),(0 1) (0 2) (3 0 1)                     ⍝ 0-3D, complex, quaternions, pga
    
    VS ← {                                                                ⍝ Vector Space
        _ ← Assert (⊂⍬) ∧.≡ ⍴¨ a b ← ⍺                                    ⍝   a and b are scalars
        _ ← Assert (⊂,1) ∧.≥ ⍴∘⍴¨ u v w ← ⍵                               ⍝   u, v and w are vectors (rank 1)
        _ ← Assert v               E v + 0                                ⍝   identity element for addition
        _ ← Assert v               E 1 × v                                ⍝   identity element for multiplication
        _ ← Assert 0               E v +_V -v                             ⍝   inverse element for addition
        _ ← Assert (v +_V w)       E w +_V v                              ⍝   commutative addition
        _ ← Assert (a × v)         E v × a                                ⍝   commutative product with scalars
        _ ← Assert (u +_V v +_V w) E w +_V u +_V v                        ⍝   associative addition
        _ ← Assert (v × a × b)     E (v × a) × b                          ⍝   associative product with scalars
        _ ← Assert (a × v +_V w)   E v +_V⍥(a∘×) w                        ⍝   distributive over addition of vectors
            Assert (v × a + b)     E a +_V⍥(v∘×) b                        ⍝   distributive over addition of scalars
    }
    _GA ← {                                                               ⍝ Geometric Algebra (geometric product ⍺⍺)
        _ ← Assert (a b ← ⍺)        VS (u v w ← ⍵)                        ⍝   vector space: a b scalars, u v w vectors
        _ ← Assert (u ⍺⍺ v ⍺⍺ w)    E w ⍺⍺⍨ u ⍺⍺ v                        ⍝   associative product
        _ ← Assert (u∘⍺⍺ ⊃+_V/ v w) E (⊃+_V/ u∘⍺⍺¨ v w)                   ⍝   distributive by left
        _ ← Assert (⍺⍺∘u ⊃+_V/ v w) E (⊃+_V/ ⍺⍺∘u¨ v w)                   ⍝   distributive by right
        _ ← Assert (a ⍺⍺ v ⍺⍺ b)    E a × b × v                           ⍝   extension of scalar product
            Assert (⊢E⍬⍴⊢)∘(⍺⍺⍨)¨   V¨BV⌈/D¨⍵                             ⍝   scalar product of parallel vectors
    }
:EndNamespace                                                             ⍝ jgl@dyalog.com 2024

In [88]:
]LINK.Export # . -overwrite  ⍝ save ga.apln file

Exported: # → C:\Users\34696\jupyter\ga